In [0]:
%%sh

aws s3 rm s3://b2c-prod-data-pipeline-unified-store-paid/unified/store.app-est-dna-log-pre-aggr.v1/fact/granularity=weekly/date=2020-06-28/ --recursive

In [0]:


from pyspark.sql.functions import *

df_2 = spark.read.format("delta").load("s3://b2c-prod-data-pipeline-unified-dna/unified/dna.publisher-company-mapping.v1/dimension/").filter("update_date = '2020-07-07'").drop("_identifier", "update_date")
df_1 = spark.read.format("delta").load("s3://b2c-prod-data-pipeline-unified-dna/unified/dna.publisher-company-mapping.v1/dimension/").filter("update_date = '2020-06-30'").drop("_identifier", "update_date")
df_3 = spark.read.format("delta").load("s3://b2c-prod-data-pipeline-unified-dna/unified/dna.mapping-log.v1/dimension/").filter("update_date = '2020-07-07'").drop("_identifier", "update_date")
df_2.createOrReplaceTempView("df2")
df_1.createOrReplaceTempView("df1")
df_3.createOrReplaceTempView("df3")


# added_df = df_2.select("publisher_id").subtract(df_1.select("publisher_id"))


diff_df_ = spark.sql("""
select * from (
select df2.*, df1.company_id as df1_company_id, df1.publisher_id as df1_publisher_id
from
df2 full outer join df1
on df2.publisher_id = df1.publisher_id
) tmp
where company_id != df1_company_id or df1_company_id is null
""")

#print diff_df_.count()
#print diff_df_.show(400, False)
diff_df_.createOrReplaceTempView("diff_df_")

diff_df = spark.sql("""
select 
df3.*
from df3 inner join diff_df_ on df3.publisher_id = diff_df_.publisher_id and df3.company_id = diff_df_.company_id
""").na.fill({"end_date": "2020-07-07"}).withColumn("start_date", when(col("start_date") == "1970-01-01", "2010-07-01").otherwise(col("start_date")))


print diff_df.count()
diff_df.show(20, False)
# diff_df_100apps = diff_df.select("app_id").distinct().orderBy("app_id").limit(100).createOrReplaceTempView("diff_df_100apps")
diff_df.createOrReplaceTempView("diff_df")
# diff_df_100apps_data = spark.sql("""select a.* from diff_df a join diff_df_100apps b on a.app_id = b.app_id""")

# diff_df_100apps_data.createOrReplaceTempView("diff_df")
# print diff_df_100apps_data.count()
# print diff_df_100apps_data.select("app_id").distinct().count()


In [0]:


from delta.tables import *
# Convert unpartitioned parquet table at path '/path/to/table'
# deltaTable = DeltaTable.convertToDelta(spark, "parquet.`/path/to/table`")
# Convert partitioned parquet table at path '/path/to/table' and partitioned by integer column named 'part'
partitionedDeltaTable = DeltaTable.convertToDelta(spark, "parquet.`s3://b2c-prod-data-pipeline-unified-store-paid/_obselete/store.app-est-dna-log-pre-aggr.v2/fact`", "granularity string, date date, device_code string")

# %sh

# aws s3 ls s3://b2c-prod-data-pipeline-unified-store-paid/_obselete/store.download-attribution-dna-log-pre-aggr.v1/fact/granularity=monthly/ --recursive

In [0]:
%%sh
# aws s3 ls s3://b2c-prod-data-pipeline-unified-store-paid/_obselete/store.app-est-dna-log.v1/fact/granularity=daily/date=2016-09-30 --recursive | wc -l  
# aws s3 ls s3://b2c-prod-data-pipeline-unified-store-paid/_obselete/store.download-attribution-dna-log-pre-aggr.v1/fact/granularity=monthly/date=2017-12-31/ --recursive |  wc -l

# aws s3 ls  s3://b2c-prod-data-pipeline-unified-store-paid/unified/store.download-attribution-dna-log.v1/fact/granularity=daily/
# aws s3 rm s3://b2c-prod-data-pipeline-unified-store-paid/_obselete/store.app-est-dna-log-pre-aggr.v2/fact/granularity=weekly/ --recursive
# aws s3 cp s3://b2c-prod-data-pipeline-unified-store-paid/unified/store.app-est-dna-log-pre-aggr.v1/fact/granularity=weekly/ s3://b2c-prod-data-pipeline-unified-store-paid/_obselete/store.app-est-dna-log-pre-aggr.v2/fact/granularity=weekly/ --recursive

aws s3 ls s3://b2c-prod-data-pipeline-unified-store-paid/_obselete/store.download-attribution-dna-log-pre-aggr.v2/fact/granularity=weekly/date=2017 --recursive

# aws s3 ls s3://b2c-prod-data-pipeline-unified-store-paid/_obselete/store.app-est-dna-log-pre-aggr.v2/fact/granularity=monthly/ --recursive

In [0]:

__df = spark.read.format("delta").load("s3://b2c-prod-data-pipeline-unified-store-paid/_obselete/store.app-est-dna-log-pre-aggr.v1/fact/").filter("granularity = 'weekly'")
__df.groupBy("date").count().orderBy("date").show(200, False)

In [0]:

daily_df = spark.read.format("delta").load("s3://b2c-prod-data-pipeline-unified-store-paid/unified/store.app-est-dna-log.v1/fact/").filter("granularity = 'daily' and date between '2017-01-01' and '2019-01-01'").drop("publisher_id", "company_id", "parent_company_id")
daily_df.createOrReplaceTempView("daily_df")


da_daily_df = spark.read.format("delta").load("s3://b2c-prod-data-pipeline-unified-store-paid/unified/store.download-attribution-dna-log.v1/fact/").filter("granularity = 'daily' and date between '2017-01-01' and '2019-01-01'").drop("publisher_id", "company_id", "parent_company_id").cache()
da_daily_df.createOrReplaceTempView("da_daily_df")


insert_daily_df = spark.sql("""
select 
    a.date,a.country_code, a.device_code, a.app_id,a.granularity,
    a.publisher_id, a.company_id,a.parent_company_id,
    a.est_free_app_download, a.est_paid_app_download, a.est_revenue,
    coalesce(b.est_paid_download, a.est_paid_download) as est_paid_download,     
    IF (
        a.est_free_app_download IS NULL AND a.est_paid_app_download IS NULL,
        NULL,
        (COALESCE(a.est_free_app_download,0) + COALESCE(a.est_paid_app_download,0)) - coalesce(b.est_paid_download, a.est_paid_download)
    ) AS est_organic_download
from (
    select daily_df.*, publisher_id, company_id, parent_company_id
    from diff_df join daily_df
    on diff_df.app_id = daily_df.app_id
    and daily_df.date between diff_df.start_date and diff_df.end_date
) a
left join (
   select date, country_code,device_code,app_id,  est_paid_download
   from da_daily_df
)b
on a.date=b.date and a.device_code=b.device_code and a.country_code = b.country_code and a.app_id=b.app_id
""").cache()

insert_daily_df.createOrReplaceTempView("insert_daily_df")
# print insert_daily_df.where("est_paid_download > 0").show(2, False)
print insert_daily_df.count()

In [0]:




# insert_weekly_df.unpersist()
insert_weekly_df = spark.sql("""
select app_id, weekday as date, country_code, device_code, publisher_id, company_id, parent_company_id, 'weekly' as granularity, SUM(est_free_app_download) AS est_free_app_download,
                        SUM(est_paid_app_download) AS est_paid_app_download,
                        SUM(est_revenue) AS est_revenue,
                        SUM(est_organic_download) AS est_organic_download,
                        SUM(est_paid_download) AS est_paid_download
from (
select insert_daily_df.*, DATE_ADD(date, 7 - DAYOFWEEK(date)) as weekday
from insert_daily_df
) tmp
group by app_id, weekday, country_code, device_code, publisher_id, company_id, parent_company_id
""").cache()
print insert_weekly_df.count()
insert_weekly_df.createOrReplaceTempView("insert_weekly_df")

In [0]:





# insert_monthly_df.unpersist()
insert_monthly_df = spark.sql("""
select app_id, monthday as date, country_code, device_code, publisher_id, company_id, parent_company_id, 'monthly' as granularity, SUM(est_free_app_download) AS est_free_app_download,
                        SUM(est_paid_app_download) AS est_paid_app_download,
                        SUM(est_revenue) AS est_revenue,
                        SUM(est_organic_download) AS est_organic_download,
                        SUM(est_paid_download) AS est_paid_download
from (
select insert_daily_df.*,last_day(date) as monthday
from insert_daily_df
) tmp
group by app_id, monthday, country_code, device_code, publisher_id, company_id, parent_company_id
""").cache()
print insert_monthly_df.count()
insert_monthly_df.createOrReplaceTempView("insert_monthly_df")
# insert_daily_df.where("est_paid_download > 0").count()

In [0]:

insert_daily_df.printSchema()

In [0]:

from delta.tables import *
from pyspark.sql.functions import *


est_table = DeltaTable.forPath(spark, "s3://b2c-prod-data-pipeline-unified-store-paid/_obselete/store.download-attribution-dna-log.v1/fact/")
insert_daily_df_da = spark.sql("""
select a.*,'daily' as granularity  from insert_daily_df a
join (
select app_id,country_code,device_code,date
from da_daily_df
group by app_id,country_code,device_code,date
) b
on a.date=b.date and a.app_id = b.app_id and a.country_code=b.country_code and a.device_code = b.device_code

""") 
est_table.alias("est").merge(
    insert_daily_df_da.alias("updates"),
    "est.app_id = updates.app_id and est.country_code = updates.country_code and est.device_code=updates.device_code and est.granularity = updates.granularity and est.granularity = 'daily' and est.date = updates.date and est.date between '2017-01-01' and '2017-01-31'") \
  .whenMatchedUpdate(set = { "company_id" : "updates.company_id", "parent_company_id": "updates.parent_company_id", "publisher_id": "updates.publisher_id" } ) \
  .execute()
  
  

In [0]:

from delta.tables import *
from pyspark.sql.functions import *
from pyspark.sql.functions import *
spark.conf.set("spark.sql.parquet.compression.codec", "gzip")
spark.conf.set("spark.delta.merge.repartitionBeforeWrite", True)
spark.conf.set("spark.sql.shuffle.partitions", 324)

######################  da
est_table = DeltaTable.forPath(spark, "s3://b2c-prod-data-pipeline-unified-store-paid/_obselete/store.download-attribution-dna-log-pre-aggr.v2/fact/")
#df_100 = df.limit(100)
#df_100.show()

insert_weekly_df_da = spark.sql("""
select a.*  from insert_weekly_df a
join (
select app_id,country_code,device_code
from da_daily_df
group by app_id,country_code,device_code
) b
on a.app_id = b.app_id and a.country_code=b.country_code and a.device_code = b.device_code

""") 
# insert_weekly_df_da.show(2,False)
est_table.alias("est").merge(
    insert_weekly_df_da.alias("updates"),
    "est.app_id = updates.app_id and est.country_code = updates.country_code and est.device_code=updates.device_code and est.granularity = updates.granularity and est.granularity = 'weekly' and est.date = updates.date and est.date between '2017-01-01' and '2019-01-01'") \
  .whenMatchedDelete() \
  .execute()
  
  
############################  app est  
est_table = DeltaTable.forPath(spark, "s3://b2c-prod-data-pipeline-unified-store-paid/_obselete/store.app-est-dna-log-pre-aggr.v2/fact/")
#df_100 = df.limit(100)

# insert_weekly_df_da.show(2,False)
est_table.alias("est").merge(
    insert_weekly_df.alias("updates"),
    "est.app_id = updates.app_id and est.country_code = updates.country_code and est.device_code=updates.device_code and est.granularity = updates.granularity and est.granularity = 'weekly' and est.date = updates.date and est.date between '2017-01-01' and '2019-01-01'") \
  .whenMatchedDelete() \
  .execute()

In [0]:

from delta.tables import *
from pyspark.sql.functions import *
from pyspark.sql.functions import *
spark.conf.set("spark.sql.parquet.compression.codec", "gzip")
spark.conf.set("spark.delta.merge.repartitionBeforeWrite", True)


est_table = DeltaTable.forPath(spark, "s3://b2c-prod-data-pipeline-unified-store-paid/_obselete/store.download-attribution-dna-log-pre-aggr.v1/fact/")
#df_100 = df.limit(100)
#df_100.show()

insert_monthly_df_da = spark.sql("""
select a.*  from insert_monthly_df a
join (
select app_id,country_code,device_code
from da_daily_df
group by app_id,country_code,device_code
) b
on a.app_id = b.app_id and a.country_code=b.country_code and a.device_code = b.device_code

""") 
# insert_weekly_df_da.where("est_paid_download > 0 ").count()
est_table.alias("est").merge(
    insert_monthly_df_da.alias("updates"),
    "est.app_id = updates.app_id and est.country_code = updates.country_code and est.device_code=updates.device_code and est.granularity = updates.granularity and est.granularity = 'monthly' and est.date = updates.date and est.date between '2017-01-01' and '2017-01-31'") \
  .whenMatchedDelete() \
  .execute()

In [0]:

spark.conf.set("spark.sql.shuffle.partitions", 1)
#print insert_df.count()
(
   insert_weekly_df_da
   .write.format("delta")
   .mode("append")
    .partitionBy(["granularity", "date", "device_code"])
    .save("s3://b2c-prod-data-pipeline-unified-store-paid/_obselete/store.download-attribution-dna-log-pre-aggr.v2/fact/")
)

spark.conf.set("spark.sql.shuffle.partitions", 1)
#print insert_df.count()
(
   insert_weekly_df
   .write.format("delta")
   .mode("append")
    .partitionBy(["granularity", "date", "device_code"])
    .save("s3://b2c-prod-data-pipeline-unified-store-paid/_obselete/store.app-est-dna-log-pre-aggr.v2/fact/")
)

In [0]:

spark.conf.set("spark.sql.shuffle.partitions", 72)
#print insert_df.count()
(
   insert_monthly_df_da
   .write.format("delta")
   .mode("append")
    .partitionBy(["granularity", "date", "device_code"])
    .save("s3://b2c-prod-data-pipeline-unified-store-paid/_obselete/store.download-attribution-dna-log-pre-aggr.v1/fact/")
)

In [0]:

insert_weekly_df_da.filter("app_id = 20600006389088").show(50, False)

In [0]:

# __df = spark.read.format("delta").load("s3://b2c-prod-data-pipeline-unified-store-paid/_obselete/store.app-est-dna-log-pre-aggr.v1/fact/").filter("granularity = 'weekly' and app_id = 1364480289 and date between '2018-08-07' and '2019-01-01'")
# __df.filter("publisher_id != 20200002612783 or company_id != 1000200000272584").show(200, False)

print spark.read.format("delta").load("s3://b2c-prod-data-pipeline-unified-store-paid/unified/store.app-est-dna-log-pre-aggr.v1/fact/").filter("granularity = 'quarterly' ").select("company_id").distinct().count()
# __df.filter("publisher_id != 20200002612783 or company_id != 1000200000272584").show(200, False)

In [0]:

storage_definition = {
    "engine": "RDB",
    "database": "aa_store_db",
    "cluster": "prod-citus-cluster-4",
    "table": "store_est_t_w_temp_fact_v3",
    "fields": [
        {"name": "app_id", "type": "bigint", "constraints": "NOT NULL"},
        {"name": "country_code", "type": "character(2)", "constraints": "NOT NULL"},
        {"name": "device_code", "type": "character varying(50)", "constraints": "NOT NULL"},
        {"name": "date", "type": "date", "format": "%Y-%m-%d", "constraints": "NOT NULL"},
        {"name": "granularity", "type": "character varying(8)", "constraints": "NOT NULL"},
        {"name": "publisher_id", "type": "bigint", "constraints": "NOT NULL"},
        {"name": "company_id", "type": "bigint", "constraints": "NOT NULL"},
        {"name": "parent_company_id", "type": "bigint", "constraints": "NOT NULL"},
        {"name": "est_free_app_download", "type": "int"},
        {"name": "est_paid_app_download", "type": "int"},
        {"name": "est_revenue", "type": "int"},
        {"name": "est_organic_download", "type": "int"},
        {"name": "est_paid_download", "type": "int"}
    ],
    "sharding_count": "64",
    "sharding_key": ["app_id"],
    "index": [
        {
            "name": "_primary_key",
            "unique": True,
            "method": "btree",
            "column": ["granularity", "device_code", "country_code", "date", "app_id", "publisher_id", "company_id", "parent_company_id"]
        },
        {
            "name": "single_app_device_first",
            "method": "btree",
            "column": ["granularity", "app_id", "date", "device_code", "country_code",
                       "est_free_app_download", "est_paid_app_download", "est_revenue", "est_organic_download",
                       "est_paid_download"]
        },
        {
            "name": "single_app_country_first",
            "method": "btree",
            "column": ["granularity", "app_id", "date", "country_code", "device_code",
                       "est_free_app_download", "est_paid_app_download", "est_revenue", "est_organic_download",
                       "est_paid_download"]
        },
        {
            "name": "join_country_first",
            "method": "btree",
            "column": ["granularity", "date", "country_code", "device_code", "app_id",
                       "est_free_app_download", "est_paid_app_download", "est_revenue", "est_organic_download",
                       "est_paid_download"],
            "include_without_query": "publisher_id, company_id, parent_company_id"
        },
        {
            "name": "join_device_first",
            "method": "btree",
            "column": ["granularity", "date", "device_code", "country_code", "app_id",
                       "est_free_app_download", "est_paid_app_download", "est_revenue", "est_organic_download",
                       "est_paid_download"],
            "include_without_query": "publisher_id, company_id, parent_company_id"
        }

    ],
    "partition_by": [{
        "name": "date",
        "type": "range",
        "granularity": "yearly"
    }],
    "action_on_conflict": "UPDATE"
}

In [0]:

from pyspark.sql.functions import *
#rows = insert_df.filter("app_id in (20600012415852)").select("date", "app_id").distinct().groupBy("app_id").agg(collect_set("date")).collect()
#insert_weekly_df = insert_weekly_df.cache()
rows = insert_weekly_df.select("date", "app_id").distinct().groupBy("app_id").agg(collect_set("date")).collect()

app_statms = []
for row in rows:
    row_dict = row.asDict()
    #print row_dict
    dates = [i.strftime('%Y-%m-%d') for i in row_dict['collect_set(date)']]
    dates_template = "".join("'{}', " * len(dates)).format(*dates)[:-2]
    #print dates_template
    app_statms.append("(app_id = {} and date in ({}))".format(row_dict["app_id"], dates_template))
#print app_statms
delete_sql = " or ".join(app_statms)
delete_sql = "granularity = 'weekly' and ({})".format(delete_sql)
print delete_sql

In [0]:

spark.read.parquet("s3://b2c-prod-data-pipeline-unified-store-paid/_obselete/store.app_est_20170101_20190101_dna_2020_0630_0707/fact/").count()

In [0]:
%%sh
PGPASSWORD='PZQYyjRHCXZ23LVQ' psql -h 10.2.6.141  -U citus_bdp_prod_int_datapipeline -d aa_store_db -p 5432 << EOF
BEGIN;

set citus.propagate_set_commands = 'local';
set LOCAL max_parallel_workers = 16;
set LOCAL max_parallel_workers_per_gather = 2;

select *
from store.store_est_t_w_temp_fact_v3
where
granularity = 'weekly' and ((app_id = 1083974444 and date in ('2017-05-06', '2017-04-15', '2018-12-08', '2018-11-17', '2018-10-27', '2017-10-28', '2017-10-07', '2017-09-16', '2018-03-31', '2018-03-10', '2017-03-11', '2017-02-18', '2017-01-28', '2018-09-22', '2018-09-01', '2017-08-12', '2017-07-22', '2018-02-03', '2018-01-13', '2018-07-28', '2018-07-07', '2017-06-17', '2017-05-27', '2018-12-29', '2017-12-09', '2017-11-18', '2018-06-02', '2018-05-12', '2018-04-21', '2017-04-22', '2017-04-01', '2018-11-03', '2018-10-13', '2017-10-14', '2017-09-23', '2017-09-02', '2018-03-17', '2018-02-24', '2017-02-04', '2017-01-14', '2018-09-08', '2018-08-18', '2017-07-29', '2017-07-08', '2018-01-20', '2017-12-30', '2018-07-14', '2018-06-23', '2017-06-03', '2017-05-13', '2019-01-05', '2018-12-15', '2018-11-24', '2017-11-25', '2017-11-04', '2018-05-19', '2018-04-28', '2018-04-07', '2017-04-08', '2017-03-18', '2017-02-25', '2018-10-20', '2018-09-29', '2017-09-30', '2017-09-09', '2017-08-19', '2018-03-03', '2018-02-10', '2017-01-21', '2018-08-25', '2018-08-04', '2017-07-15', '2017-06-24', '2018-01-06', '2017-12-16', '2018-06-30', '2018-06-09', '2017-05-20', '2017-04-29', '2018-12-22', '2018-12-01', '2018-11-10', '2017-11-11', '2017-10-21', '2018-05-05', '2018-04-14', '2018-03-24', '2017-03-25', '2017-03-04', '2017-02-11', '2018-10-06', '2018-09-15', '2017-08-26', '2017-08-05', '2018-02-17', '2018-01-27', '2017-01-07', '2018-08-11', '2018-07-21', '2017-07-01', '2017-06-10', '2017-12-23', '2017-12-02', '2018-06-16', '2018-05-26')) or (app_id = 787292498 and date in ('2017-02-04')) or (app_id = 532216547 and date in ('2018-07-21', '2017-03-18', '2018-07-28')) or (app_id = 556435057 and date in ('2017-07-22', '2018-11-03')) or (app_id = 20600006230637 and date in ('2017-05-06', '2017-04-15', '2017-04-22', '2017-04-01', '2017-09-16', '2017-04-08', '2017-09-23', '2017-03-25', '2017-09-02', '2017-01-14', '2017-07-22', '2017-06-24', '2017-06-10', '2017-06-17', '2017-05-27', '2017-06-03', '2017-05-13', '2017-05-20', '2017-04-29', '2018-05-26')) or (app_id = 1410408871 and date in ('2018-12-08', '2018-09-15', '2018-12-29', '2018-12-15', '2018-11-24')) or (app_id = 1379847700 and date in ('2018-12-22', '2018-12-01', '2018-11-10', '2018-12-08', '2018-11-17', '2018-10-27', '2018-11-03', '2018-10-13', '2018-10-20', '2018-10-06', '2018-12-29', '2019-01-05', '2018-12-15', '2018-11-24')) or (app_id = 20600006880641 and date in ('2017-05-06', '2017-10-28', '2017-10-07', '2017-09-16', '2018-03-31', '2018-03-10', '2017-08-12', '2017-07-22', '2018-02-03', '2018-01-13', '2018-07-28', '2018-07-07', '2017-06-17', '2017-05-27', '2017-12-09', '2017-11-18', '2018-06-02', '2018-05-12', '2018-04-21', '2017-10-14', '2017-09-23', '2017-09-02', '2018-03-17', '2018-02-24', '2018-08-18', '2017-07-29', '2017-07-08', '2018-01-20', '2017-12-30', '2018-07-14', '2018-06-23', '2017-06-03', '2017-05-13', '2017-11-25', '2017-11-04', '2018-05-19', '2018-04-28', '2018-04-07', '2017-09-30', '2017-09-09', '2017-08-19', '2018-03-03', '2018-02-10', '2018-08-04', '2017-07-15', '2017-06-24', '2018-01-06', '2017-12-16', '2018-06-30', '2018-06-09', '2017-05-20', '2017-04-29', '2017-11-11', '2017-10-21', '2018-05-05', '2018-04-14', '2018-03-24', '2017-08-26', '2017-08-05', '2018-02-17', '2018-01-27', '2018-08-11', '2018-07-21', '2017-07-01', '2017-06-10', '2017-12-23', '2017-12-02', '2018-06-16', '2018-05-26')) or (app_id = 20600008253949 and date in ('2018-12-22', '2018-12-01', '2018-12-29', '2019-01-05')) or (app_id = 20600004791770 and date in ('2018-12-22', '2018-12-01', '2018-11-10', '2018-12-08', '2018-11-17', '2018-10-27', '2018-11-03', '2018-10-13', '2018-10-06', '2018-09-08', '2018-07-28', '2018-12-29', '2019-01-05', '2018-12-15', '2018-11-24')) or (app_id = 20600007028334 and date in ('2017-05-06', '2017-04-15', '2018-12-08', '2018-11-17', '2018-10-27', '2017-10-28', '2017-10-07', '2017-09-16', '2018-03-31', '2018-03-10', '2018-09-22', '2018-09-01', '2017-08-12', '2017-07-22', '2018-02-03', '2018-01-13', '2018-07-28', '2018-07-07', '2017-06-17', '2017-05-27', '2018-12-29', '2017-12-09', '2017-11-18', '2018-06-02', '2018-05-12', '2018-04-21', '2017-04-22', '2017-04-01', '2018-11-03', '2018-10-13', '2017-10-14', '2017-09-23', '2017-09-02', '2018-03-17', '2018-02-24', '2018-09-08', '2018-08-18', '2017-07-29', '2017-07-08', '2018-01-20', '2017-12-30', '2018-07-14', '2018-06-23', '2017-06-03', '2017-05-13', '2019-01-05', '2018-12-15', '2018-11-24', '2017-11-25', '2017-11-04', '2018-05-19', '2018-04-28', '2018-04-07', '2017-04-08', '2018-10-20', '2018-09-29', '2017-09-30', '2017-09-09', '2017-08-19', '2018-03-03', '2018-02-10', '2018-08-25', '2018-08-04', '2017-07-15', '2017-06-24', '2018-01-06', '2017-12-16', '2018-06-30', '2018-06-09', '2017-05-20', '2017-04-29', '2018-12-22', '2018-12-01', '2018-11-10', '2017-11-11', '2017-10-21', '2018-05-05', '2018-04-14', '2018-03-24', '2017-03-25', '2018-10-06', '2018-09-15', '2017-08-26', '2017-08-05', '2018-02-17', '2018-01-27', '2018-08-11', '2018-07-21', '2017-07-01', '2017-06-10', '2017-12-23', '2017-12-02', '2018-06-16', '2018-05-26')) or (app_id = 417432672 and date in ('2017-05-06', '2017-04-15', '2018-12-08', '2018-11-17', '2018-10-27', '2017-10-28', '2017-10-07', '2017-09-16', '2018-03-31', '2018-03-10', '2017-03-11', '2017-02-18', '2017-01-28', '2018-09-22', '2018-09-01', '2017-08-12', '2017-07-22', '2018-02-03', '2018-01-13', '2018-07-28', '2018-07-07', '2017-06-17', '2017-05-27', '2018-12-29', '2017-12-09', '2017-11-18', '2018-06-02', '2018-05-12', '2018-04-21', '2017-04-22', '2017-04-01', '2018-11-03', '2018-10-13', '2017-10-14', '2017-09-23', '2017-09-02', '2018-03-17', '2018-02-24', '2017-02-04', '2017-01-14', '2018-09-08', '2018-08-18', '2017-07-29', '2017-07-08', '2018-01-20', '2017-12-30', '2018-07-14', '2018-06-23', '2017-06-03', '2017-05-13', '2019-01-05', '2018-12-15', '2018-11-24', '2017-11-25', '2017-11-04', '2018-05-19', '2018-04-28', '2018-04-07', '2017-04-08', '2017-03-18', '2017-02-25', '2018-10-20', '2018-09-29', '2017-09-30', '2017-09-09', '2017-08-19', '2018-03-03', '2018-02-10', '2017-01-21', '2018-08-25', '2018-08-04', '2017-07-15', '2017-06-24', '2018-01-06', '2017-12-16', '2018-06-30', '2018-06-09', '2017-05-20', '2017-04-29', '2018-12-22', '2018-12-01', '2018-11-10', '2017-11-11', '2017-10-21', '2018-05-05', '2018-04-14', '2018-03-24', '2017-03-25', '2017-03-04', '2017-02-11', '2018-10-06', '2018-09-15', '2017-08-26', '2017-08-05', '2018-02-17', '2018-01-27', '2017-01-07', '2018-08-11', '2018-07-21', '2017-07-01', '2017-06-10', '2017-12-23', '2017-12-02', '2018-06-16', '2018-05-26')) or (app_id = 20600007880532 and date in ('2018-12-08', '2018-11-17', '2018-10-27', '2017-10-28', '2017-10-07', '2017-09-16', '2018-03-31', '2018-03-10', '2018-09-22', '2018-09-01', '2017-08-12', '2018-02-03', '2018-01-13', '2018-07-28', '2018-07-07', '2018-12-29', '2017-12-09', '2017-11-18', '2018-06-02', '2018-05-12', '2018-04-21', '2018-11-03', '2018-10-13', '2017-10-14', '2017-09-23', '2017-09-02', '2018-03-17', '2018-02-24', '2018-09-08', '2018-08-18', '2018-01-20', '2017-12-30', '2018-07-14', '2018-06-23', '2019-01-05', '2018-12-15', '2018-11-24', '2017-11-25', '2017-11-04', '2018-05-19', '2018-04-28', '2018-04-07', '2018-10-20', '2018-09-29', '2017-09-30', '2017-09-09', '2017-08-19', '2018-03-03', '2018-02-10', '2018-08-25', '2018-08-04', '2018-01-06', '2017-12-16', '2018-06-30', '2018-06-09', '2018-12-22', '2018-12-01', '2018-11-10', '2017-11-11', '2017-10-21', '2018-05-05', '2018-04-14', '2018-03-24', '2018-10-06', '2018-09-15', '2017-08-26', '2017-08-05', '2018-02-17', '2018-01-27', '2018-08-11', '2018-07-21', '2017-12-23', '2017-12-02', '2018-06-16', '2018-05-26')) or (app_id = 1086538905 and date in ('2018-12-22', '2018-12-01', '2018-12-08', '2018-12-29', '2019-01-05', '2018-12-15', '2018-11-24')) or (app_id = 20600004833037 and date in ('2018-05-26', '2018-12-08', '2018-11-17', '2018-05-19', '2018-04-28', '2018-10-27', '2018-10-20', '2018-09-29', '2018-09-22', '2018-09-01', '2018-08-25', '2018-08-04', '2018-07-28', '2018-07-07', '2018-12-29', '2018-06-30', '2018-06-09', '2018-12-22', '2018-12-01', '2018-11-10', '2018-05-12', '2018-06-02', '2018-11-03', '2018-05-05', '2018-10-13', '2018-10-06', '2018-09-15', '2018-09-08', '2018-08-18', '2018-08-11', '2018-07-21', '2018-07-14', '2018-06-23', '2019-01-05', '2018-12-15', '2018-06-16', '2018-11-24')) or (app_id = 20600005946827 and date in ('2017-05-06', '2017-04-15', '2017-04-22', '2017-04-01', '2017-04-08', '2017-03-18', '2017-02-25', '2017-03-25', '2017-03-04', '2017-02-11', '2017-03-11', '2017-02-18', '2017-01-28', '2017-02-04', '2017-01-14', '2017-01-21', '2017-01-07', '2017-05-13', '2017-04-29')) or (app_id = 1262256275 and date in ('2018-12-08', '2018-11-17', '2018-10-27', '2017-10-28', '2017-10-07', '2017-09-16', '2018-03-31', '2018-03-10', '2018-09-22', '2018-09-01', '2017-08-12', '2018-02-03', '2018-01-13', '2018-07-28', '2018-07-07', '2018-12-29', '2017-12-09', '2017-11-18', '2018-06-02', '2018-05-12', '2018-04-21', '2018-11-03', '2018-10-13', '2017-10-14', '2017-09-23', '2017-09-02', '2018-03-17', '2018-02-24', '2018-09-08', '2018-08-18', '2018-01-20', '2017-12-30', '2018-07-14', '2018-06-23', '2019-01-05', '2018-12-15', '2018-11-24', '2017-11-25', '2017-11-04', '2018-05-19', '2018-04-28', '2018-04-07', '2018-10-20', '2018-09-29', '2017-09-30', '2017-09-09', '2017-08-19', '2018-03-03', '2018-02-10', '2018-08-25', '2018-08-04', '2018-01-06', '2017-12-16', '2018-06-30', '2018-06-09', '2018-12-22', '2018-12-01', '2018-11-10', '2017-11-11', '2017-10-21', '2018-05-05', '2018-04-14', '2018-03-24', '2018-10-06', '2018-09-15', '2017-08-26', '2018-02-17', '2018-01-27', '2018-08-11', '2018-07-21', '2017-12-23', '2017-12-02', '2018-06-16', '2018-05-26')) or (app_id = 20600010089491 and date in ('2018-10-27')) or (app_id = 20600003413158 and date in ('2017-05-06', '2017-04-15', '2018-12-08', '2018-11-17', '2018-10-27', '2017-10-28', '2017-10-07', '2017-09-16', '2018-03-31', '2018-03-10', '2017-03-11', '2017-02-18', '2017-01-28', '2018-09-22', '2018-09-01', '2017-08-12', '2017-07-22', '2018-02-03', '2018-01-13', '2018-07-28', '2018-07-07', '2017-06-17', '2017-05-27', '2018-12-29', '2017-12-09', '2017-11-18', '2018-06-02', '2018-05-12', '2018-04-21', '2017-04-22', '2017-04-01', '2018-11-03', '2018-10-13', '2017-10-14', '2017-09-23', '2017-09-02', '2018-03-17', '2018-02-24', '2017-02-04', '2017-01-14', '2018-09-08', '2018-08-18', '2017-07-29', '2017-07-08', '2018-01-20', '2017-12-30', '2018-07-14', '2018-06-23', '2017-06-03', '2017-05-13', '2019-01-05', '2018-12-15', '2018-11-24', '2017-11-25', '2017-11-04', '2018-05-19', '2018-04-28', '2018-04-07', '2017-04-08', '2017-03-18', '2017-02-25', '2018-10-20', '2018-09-29', '2017-09-30', '2017-09-09', '2017-08-19', '2018-03-03', '2018-02-10', '2017-01-21', '2018-08-25', '2018-08-04', '2017-07-15', '2017-06-24', '2018-01-06', '2017-12-16', '2018-06-30', '2018-06-09', '2017-05-20', '2017-04-29', '2018-12-22', '2018-12-01', '2018-11-10', '2017-11-11', '2017-10-21', '2018-05-05', '2018-04-14', '2018-03-24', '2017-03-25', '2017-03-04', '2017-02-11', '2018-10-06', '2018-09-15', '2017-08-26', '2017-08-05', '2018-02-17', '2018-01-27', '2017-01-07', '2018-08-11', '2018-07-21', '2017-07-01', '2017-06-10', '2017-12-23', '2017-12-02', '2018-06-16', '2018-05-26')) or (app_id = 20600010842891 and date in ('2018-12-01', '2018-11-24')) or (app_id = 1029379465 and date in ('2018-12-08', '2018-11-17', '2018-10-27', '2018-10-20', '2018-09-29', '2018-09-22', '2018-09-01', '2018-08-25', '2018-08-04', '2018-07-28', '2018-07-07', '2018-12-29', '2018-06-30', '2018-12-22', '2018-12-01', '2018-11-10', '2018-11-03', '2018-10-13', '2018-10-06', '2018-09-15', '2018-09-08', '2018-08-18', '2018-08-11', '2018-07-21', '2018-07-14', '2019-01-05', '2018-12-15', '2018-11-24')) or (app_id = 20600006334674 and date in ('2018-11-24', '2018-12-08', '2018-11-17', '2018-10-27', '2018-04-28', '2018-05-19', '2018-10-20', '2018-09-29', '2018-09-22', '2018-09-01', '2018-08-25', '2018-08-04', '2018-07-28', '2018-07-07', '2018-12-29', '2018-06-30', '2018-06-09', '2018-12-22', '2018-12-01', '2018-11-10', '2018-05-12', '2018-06-02', '2018-04-21', '2018-11-03', '2018-05-05', '2018-04-14', '2018-10-13', '2018-10-06', '2018-09-15', '2018-09-08', '2018-08-18', '2018-08-11', '2018-07-21', '2018-07-14', '2018-06-23', '2019-01-05', '2018-12-15', '2018-06-16', '2018-05-26')) or (app_id = 20600009484035 and date in ('2018-06-16', '2018-12-08', '2018-11-17', '2018-10-27', '2018-05-19', '2018-04-28', '2018-04-07', '2018-10-20', '2018-09-29', '2018-03-31', '2018-09-22', '2018-09-01', '2018-08-25', '2018-08-04', '2018-07-28', '2018-07-07', '2018-12-29', '2018-06-30', '2018-06-09', '2018-12-22', '2018-12-01', '2018-11-10', '2018-05-12', '2018-06-02', '2018-04-21', '2018-11-03', '2018-05-05', '2018-04-14', '2018-10-13', '2018-10-06', '2018-09-15', '2018-09-08', '2018-08-18', '2018-08-11', '2018-07-21', '2018-07-14', '2018-06-23', '2019-01-05', '2018-12-15', '2018-11-24', '2018-05-26')) or (app_id = 391967884 and date in ('2017-05-06', '2017-04-15', '2018-12-08', '2018-11-17', '2018-10-27', '2017-10-28', '2017-10-07', '2017-09-16', '2018-03-31', '2018-03-10', '2017-03-11', '2017-02-18', '2017-01-28', '2018-09-22', '2018-09-01', '2017-08-12', '2017-07-22', '2018-02-03', '2018-01-13', '2018-07-28', '2018-07-07', '2017-06-17', '2017-05-27', '2018-12-29', '2017-12-09', '2017-11-18', '2018-06-02', '2018-05-12', '2018-04-21', '2017-04-22', '2017-04-01', '2018-11-03', '2018-10-13', '2017-10-14', '2017-09-23', '2017-09-02', '2018-03-17', '2018-02-24', '2017-02-04', '2017-01-14', '2018-09-08', '2018-08-18', '2017-07-29', '2017-07-08', '2018-01-20', '2017-12-30', '2018-07-14', '2018-06-23', '2017-06-03', '2017-05-13', '2019-01-05', '2018-12-15', '2018-11-24', '2017-11-25', '2017-11-04', '2018-05-19', '2018-04-28', '2018-04-07', '2017-04-08', '2017-03-18', '2017-02-25', '2018-10-20', '2018-09-29', '2017-09-30', '2017-09-09', '2017-08-19', '2018-03-03', '2018-02-10', '2017-01-21', '2018-08-25', '2018-08-04', '2017-07-15', '2017-06-24', '2018-01-06', '2017-12-16', '2018-06-30', '2018-06-09', '2017-05-20', '2017-04-29', '2018-12-22', '2018-12-01', '2018-11-10', '2017-11-11', '2017-10-21', '2018-05-05', '2018-04-14', '2018-03-24', '2017-03-25', '2017-03-04', '2017-02-11', '2018-10-06', '2018-09-15', '2017-08-26', '2017-08-05', '2018-02-17', '2018-01-27', '2017-01-07', '2018-08-11', '2018-07-21', '2017-07-01', '2017-06-10', '2017-12-23', '2017-12-02', '2018-06-16', '2018-05-26')) or (app_id = 1012318179 and date in ('2017-05-06', '2017-04-15', '2017-11-11', '2018-11-17', '2017-03-25', '2018-09-29', '2017-02-11', '2018-03-10', '2018-03-31', '2018-10-06', '2017-01-28', '2018-03-03', '2017-07-22', '2017-06-17', '2017-12-02', '2017-05-13', '2018-06-09')) or (app_id = 1373080886 and date in ('2018-06-16', '2018-12-08', '2018-11-17', '2018-05-19', '2018-10-27', '2018-10-20', '2018-09-29', '2018-09-22', '2018-09-01', '2018-08-25', '2018-08-04', '2018-07-28', '2018-07-07', '2018-12-29', '2018-06-30', '2018-06-09', '2018-12-22', '2018-12-01', '2018-06-02', '2018-05-12', '2018-11-10', '2018-11-03', '2018-10-13', '2018-05-05', '2018-10-06', '2018-09-15', '2018-09-08', '2018-08-18', '2018-08-11', '2018-07-21', '2018-07-14', '2018-06-23', '2019-01-05', '2018-12-15', '2018-11-24', '2018-05-26')) or (app_id = 1357091692 and date in ('2018-08-04', '2018-04-07')) or (app_id = 1209440054 and date in ('2018-02-03', '2017-06-24', '2017-07-01', '2017-06-17', '2018-02-10')) or (app_id = 20600004844447 and date in ('2017-05-06', '2017-04-15', '2018-12-08', '2018-11-17', '2018-10-27', '2017-10-28', '2017-10-07', '2017-09-16', '2018-03-31', '2018-03-10', '2017-03-11', '2017-02-18', '2017-01-28', '2018-09-22', '2018-09-01', '2017-08-12', '2017-07-22', '2018-02-03', '2018-01-13', '2018-07-28', '2018-07-07', '2017-06-17', '2017-05-27', '2018-12-29', '2017-12-09', '2017-11-18', '2018-06-02', '2018-05-12', '2018-04-21', '2017-04-22', '2017-04-01', '2018-11-03', '2018-10-13', '2017-10-14', '2017-09-23', '2017-09-02', '2018-03-17', '2018-02-24', '2017-02-04', '2017-01-14', '2018-09-08', '2018-08-18', '2017-07-29', '2017-07-08', '2018-01-20', '2017-12-30', '2018-07-14', '2018-06-23', '2017-06-03', '2017-05-13', '2019-01-05', '2018-12-15', '2018-11-24', '2017-11-25', '2017-11-04', '2018-05-19', '2018-04-28', '2018-04-07', '2017-04-08', '2017-03-18', '2017-02-25', '2018-10-20', '2018-09-29', '2017-09-30', '2017-09-09', '2017-08-19', '2018-03-03', '2018-02-10', '2017-01-21', '2018-08-25', '2018-08-04', '2017-07-15', '2017-06-24', '2017-12-16', '2018-06-30', '2018-06-09', '2017-05-20', '2017-04-29', '2018-12-22', '2018-12-01', '2018-11-10', '2017-11-11', '2017-10-21', '2018-05-05', '2018-04-14', '2018-03-24', '2017-03-25', '2017-03-04', '2017-02-11', '2018-10-06', '2018-09-15', '2017-08-26', '2017-08-05', '2018-02-17', '2018-01-27', '2017-01-07', '2018-08-11', '2018-07-21', '2017-07-01', '2017-06-10', '2017-12-23', '2017-12-02', '2018-06-16', '2018-05-26')) or (app_id = 514736092 and date in ('2017-05-06', '2017-04-15', '2018-12-08', '2018-11-17', '2018-10-27', '2017-10-28', '2017-10-07', '2017-09-16', '2018-03-31', '2018-03-10', '2017-03-11', '2017-02-18', '2017-01-28', '2018-09-22', '2018-09-01', '2017-08-12', '2017-07-22', '2018-02-03', '2018-01-13', '2018-07-28', '2018-07-07', '2017-06-17', '2017-05-27', '2018-12-29', '2017-12-09', '2017-11-18', '2018-06-02', '2018-05-12', '2018-04-21', '2017-04-22', '2017-04-01', '2018-11-03', '2018-10-13', '2017-10-14', '2017-09-23', '2017-09-02', '2018-03-17', '2018-02-24', '2017-02-04', '2017-01-14', '2018-09-08', '2018-08-18', '2017-07-29', '2017-07-08', '2018-01-20', '2017-12-30', '2018-07-14', '2018-06-23', '2017-06-03', '2017-05-13', '2019-01-05', '2018-12-15', '2018-11-24', '2017-11-25', '2017-11-04', '2018-05-19', '2018-04-28', '2018-04-07', '2017-04-08', '2017-03-18', '2017-02-25', '2018-10-20', '2018-09-29', '2017-09-30', '2017-09-09', '2017-08-19', '2018-03-03', '2018-02-10', '2017-01-21', '2018-08-25', '2018-08-04', '2017-07-15', '2017-06-24', '2018-01-06', '2017-12-16', '2018-06-30', '2018-06-09', '2017-05-20', '2017-04-29', '2018-12-22', '2018-12-01', '2018-11-10', '2017-11-11', '2017-10-21', '2018-05-05', '2018-04-14', '2018-03-24', '2017-03-25', '2017-03-04', '2017-02-11', '2018-10-06', '2018-09-15', '2017-08-26', '2017-08-05', '2018-02-17', '2018-01-27', '2017-01-07', '2018-08-11', '2018-07-21', '2017-07-01', '2017-06-10', '2017-12-23', '2017-12-02', '2018-06-16', '2018-05-26')) or (app_id = 648244118 and date in ('2018-01-27', '2018-11-17', '2017-05-13')) or (app_id = 20600005905811 and date in ('2017-12-23')) or (app_id = 20600010898232 and date in ('2018-12-22', '2018-12-01', '2018-12-08', '2018-12-29', '2019-01-05', '2018-12-15')) or (app_id = 20600008358218 and date in ('2018-06-16', '2018-12-08', '2018-11-17', '2018-10-27', '2018-04-28', '2018-04-07', '2018-05-19', '2018-10-20', '2018-09-29', '2018-03-31', '2018-03-10', '2018-09-22', '2018-09-01', '2018-03-03', '2018-02-10', '2018-08-25', '2018-08-04', '2018-02-03', '2018-07-28', '2018-12-29', '2018-06-30', '2018-06-09', '2017-11-18', '2018-12-22', '2018-12-01', '2018-11-10', '2018-05-12', '2018-04-21', '2018-06-02', '2018-11-03', '2018-10-13', '2018-04-14', '2018-05-05', '2018-03-24', '2018-10-06', '2018-09-15', '2018-03-17', '2018-02-24', '2018-09-08', '2018-08-18', '2018-02-17', '2018-01-27', '2018-08-11', '2018-07-21', '2018-07-14', '2018-06-23', '2019-01-05', '2018-12-15', '2018-11-24', '2018-05-26')) or (app_id = 1096504117 and date in ('2017-05-06', '2017-04-15', '2018-12-08', '2018-11-17', '2018-10-27', '2017-10-28', '2017-10-07', '2017-09-16', '2018-03-31', '2018-03-10', '2017-03-11', '2017-02-18', '2017-01-28', '2018-09-22', '2018-09-01', '2017-08-12', '2017-07-22', '2018-02-03', '2018-01-13', '2018-07-28', '2018-07-07', '2017-06-17', '2017-05-27', '2018-12-29', '2017-12-09', '2017-11-18', '2018-06-02', '2018-05-12', '2018-04-21', '2017-04-22', '2017-04-01', '2018-11-03', '2018-10-13', '2017-10-14', '2017-09-23', '2017-09-02', '2018-03-17', '2018-02-24', '2017-02-04', '2017-01-14', '2018-09-08', '2018-08-18', '2017-07-29', '2017-07-08', '2018-01-20', '2017-12-30', '2018-07-14', '2018-06-23', '2017-06-03', '2017-05-13', '2019-01-05', '2018-12-15', '2018-11-24', '2017-11-25', '2017-11-04', '2018-05-19', '2018-04-28', '2018-04-07', '2017-04-08', '2017-03-18', '2017-02-25', '2018-10-20', '2018-09-29', '2017-09-30', '2017-09-09', '2017-08-19', '2018-03-03', '2018-02-10', '2017-01-21', '2018-08-25', '2018-08-04', '2017-07-15', '2017-06-24', '2018-01-06', '2017-12-16', '2018-06-30', '2018-06-09', '2017-05-20', '2017-04-29', '2018-12-22', '2018-12-01', '2018-11-10', '2017-11-11', '2017-10-21', '2018-05-05', '2018-04-14', '2018-03-24', '2017-03-25', '2017-03-04', '2017-02-11', '2018-10-06', '2018-09-15', '2017-08-26', '2017-08-05', '2018-02-17', '2018-01-27', '2017-01-07', '2018-08-11', '2018-07-21', '2017-07-01', '2017-06-10', '2017-12-23', '2017-12-02', '2018-06-16', '2018-05-26')) or (app_id = 20600007244402 and date in ('2017-05-06', '2017-04-15', '2018-12-08', '2018-11-17', '2018-10-27', '2017-10-28', '2017-10-07', '2017-09-16', '2018-03-31', '2018-03-10', '2018-09-22', '2018-09-01', '2017-08-12', '2017-07-22', '2018-02-03', '2018-01-13', '2018-07-28', '2018-07-07', '2017-06-17', '2017-05-27', '2018-12-29', '2017-12-09', '2017-11-18', '2018-06-02', '2018-05-12', '2018-04-21', '2017-04-22', '2018-11-03', '2018-10-13', '2017-10-14', '2017-09-23', '2017-09-02', '2018-03-17', '2018-02-24', '2018-09-08', '2018-08-18', '2017-07-29', '2017-07-08', '2018-01-20', '2017-12-30', '2018-07-14', '2018-06-23', '2017-06-03', '2017-05-13', '2019-01-05', '2018-12-15', '2018-11-24', '2017-11-25', '2017-11-04', '2018-05-19', '2018-04-28', '2018-04-07', '2018-10-20', '2018-09-29', '2017-09-30', '2017-09-09', '2017-08-19', '2018-03-03', '2018-02-10', '2018-08-25', '2018-08-04', '2017-07-15', '2017-06-24', '2018-01-06', '2017-12-16', '2018-06-30', '2018-06-09', '2017-05-20', '2017-04-29', '2018-12-22', '2018-12-01', '2018-11-10', '2017-11-11', '2017-10-21', '2018-05-05', '2018-04-14', '2018-03-24', '2018-10-06', '2018-09-15', '2017-08-26', '2017-08-05', '2018-02-17', '2018-01-27', '2018-08-11', '2018-07-21', '2017-07-01', '2017-06-10', '2017-12-23', '2017-12-02', '2018-06-16', '2018-05-26')) or (app_id = 20600009555942 and date in ('2018-12-22', '2018-12-01', '2018-11-10', '2018-12-08', '2018-11-17', '2018-10-27', '2018-11-03', '2018-10-20', '2018-12-29', '2019-01-05', '2018-12-15', '2018-11-24')) or (app_id = 796785501 and date in ('2017-02-25')) or (app_id = 1093604517 and date in ('2018-12-08', '2018-11-17', '2018-10-27', '2018-10-20', '2018-09-29', '2018-09-22', '2018-09-01', '2018-08-25', '2018-08-04', '2018-07-28', '2018-07-07', '2018-12-29', '2018-06-30', '2018-12-22', '2018-12-01', '2018-11-10', '2018-11-03', '2018-10-13', '2018-10-06', '2018-09-15', '2018-09-08', '2018-08-18', '2018-08-11', '2018-07-21', '2018-07-14', '2019-01-05', '2018-12-15', '2018-11-24')) or (app_id = 20600005857714 and date in ('2017-11-25', '2017-11-04', '2017-05-06', '2017-04-15', '2017-10-28', '2017-10-07', '2017-04-08', '2017-09-16', '2017-02-25', '2017-03-18', '2017-09-30', '2017-09-09', '2017-08-19', '2017-03-11', '2017-02-18', '2017-01-28', '2017-08-12', '2017-07-22', '2017-01-21', '2017-07-15', '2017-06-24', '2017-12-16', '2017-06-17', '2017-05-27', '2017-12-09', '2017-11-18', '2017-05-20', '2017-04-29', '2017-11-11', '2017-10-21', '2017-04-22', '2017-04-01', '2017-10-14', '2017-09-23', '2017-09-02', '2017-03-04', '2017-02-11', '2017-03-25', '2017-08-26', '2017-08-05', '2017-02-04', '2017-01-14', '2017-07-29', '2017-07-08', '2017-01-07', '2017-07-01', '2017-06-10', '2017-12-02', '2017-06-03', '2017-05-13')) or (app_id = 20600005364159 and date in ('2018-05-26', '2018-12-08', '2018-11-17', '2018-10-27', '2018-04-28', '2018-04-07', '2017-10-07', '2017-10-28', '2018-10-20', '2018-09-29', '2018-03-31', '2018-03-10', '2017-09-09', '2017-09-30', '2018-09-22', '2018-09-01', '2018-03-03', '2018-08-25', '2018-08-04', '2018-07-28', '2018-07-07', '2017-05-27', '2018-12-29', '2018-06-30', '2018-06-09', '2018-12-22', '2018-12-01', '2018-11-10', '2018-05-12', '2018-11-03', '2018-10-13', '2018-03-24', '2017-09-23', '2017-10-14', '2018-10-06', '2018-09-15', '2018-03-17', '2018-02-24', '2018-09-08', '2018-08-18', '2018-02-17', '2017-01-07', '2018-08-11', '2018-07-21', '2018-07-14', '2018-06-23', '2019-01-05', '2018-12-15', '2018-11-24', '2018-06-16')) or (app_id = 974674099 and date in ('2017-05-06', '2017-04-15', '2018-12-08', '2018-11-17', '2018-10-27', '2017-10-28', '2017-10-07', '2017-09-16', '2018-03-31', '2018-03-10', '2017-03-11', '2017-02-18', '2017-01-28', '2018-09-22', '2018-09-01', '2017-08-12', '2017-07-22', '2018-02-03', '2018-01-13', '2018-07-28', '2018-07-07', '2017-06-17', '2017-05-27', '2018-12-29', '2017-12-09', '2017-11-18', '2018-06-02', '2018-05-12', '2018-04-21', '2017-04-22', '2017-04-01', '2018-11-03', '2018-10-13', '2017-10-14', '2017-09-23', '2017-09-02', '2018-03-17', '2018-02-24', '2017-02-04', '2017-01-14', '2018-09-08', '2018-08-18', '2017-07-29', '2017-07-08', '2018-01-20', '2017-12-30', '2018-07-14', '2018-06-23', '2017-06-03', '2017-05-13', '2019-01-05', '2018-12-15', '2018-11-24', '2017-11-25', '2017-11-04', '2018-05-19', '2018-04-28', '2018-04-07', '2017-04-08', '2017-03-18', '2017-02-25', '2018-10-20', '2018-09-29', '2017-09-30', '2017-09-09', '2017-08-19', '2018-03-03', '2018-02-10', '2017-01-21', '2018-08-25', '2018-08-04', '2017-07-15', '2017-06-24', '2018-01-06', '2017-12-16', '2018-06-30', '2018-06-09', '2017-05-20', '2017-04-29', '2018-12-22', '2018-12-01', '2018-11-10', '2017-11-11', '2017-10-21', '2018-05-05', '2018-04-14', '2018-03-24', '2017-03-25', '2017-03-04', '2017-02-11', '2018-10-06', '2018-09-15', '2017-08-26', '2017-08-05', '2018-02-17', '2018-01-27', '2017-01-07', '2018-08-11', '2018-07-21', '2017-07-01', '2017-06-10', '2017-12-23', '2017-12-02', '2018-06-16', '2018-05-26')) or (app_id = 489153186 and date in ('2017-05-06', '2017-04-15', '2018-12-08', '2018-11-17', '2018-10-27', '2017-10-28', '2017-10-07', '2017-09-16', '2018-03-31', '2018-03-10', '2017-03-11', '2017-02-18', '2017-01-28', '2018-09-22', '2018-09-01', '2017-08-12', '2017-07-22', '2018-02-03', '2018-01-13', '2018-07-28', '2018-07-07', '2017-06-17', '2017-05-27', '2018-12-29', '2017-12-09', '2017-11-18', '2018-06-02', '2018-05-12', '2018-04-21', '2017-04-22', '2017-04-01', '2018-11-03', '2018-10-13', '2017-10-14', '2017-09-23', '2017-09-02', '2018-03-17', '2018-02-24', '2017-02-04', '2017-01-14', '2018-09-08', '2018-08-18', '2017-07-29', '2017-07-08', '2018-01-20', '2017-12-30', '2018-07-14', '2018-06-23', '2017-06-03', '2017-05-13', '2019-01-05', '2018-12-15', '2018-11-24', '2017-11-25', '2017-11-04', '2018-05-19', '2018-04-28', '2018-04-07', '2017-04-08', '2017-03-18', '2017-02-25', '2018-10-20', '2018-09-29', '2017-09-30', '2017-09-09', '2017-08-19', '2018-03-03', '2018-02-10', '2017-01-21', '2018-08-25', '2018-08-04', '2017-07-15', '2017-06-24', '2018-01-06', '2017-12-16', '2018-06-30', '2018-06-09', '2017-05-20', '2017-04-29', '2018-12-22', '2018-12-01', '2018-11-10', '2017-11-11', '2017-10-21', '2018-05-05', '2018-04-14', '2018-03-24', '2017-03-25', '2017-03-04', '2017-02-11', '2018-09-15', '2017-08-26', '2017-08-05', '2018-02-17', '2018-01-27', '2017-01-07', '2018-08-11', '2018-07-21', '2017-07-01', '2017-06-10', '2017-12-23', '2017-12-02', '2018-06-16', '2018-05-26')) or (app_id = 20600009788852 and date in ('2018-06-16', '2018-12-08', '2018-11-17', '2018-10-27', '2018-10-20', '2018-09-29', '2018-09-22', '2018-09-01', '2018-08-25', '2018-08-04', '2018-07-28', '2018-07-07', '2018-12-29', '2018-06-30', '2018-06-09', '2018-12-22', '2018-12-01', '2018-11-10', '2018-06-02', '2018-11-03', '2018-10-13', '2018-10-06', '2018-09-15', '2018-09-08', '2018-08-18', '2018-08-11', '2018-07-21', '2018-07-14', '2018-06-23', '2019-01-05', '2018-12-15', '2018-11-24', '2018-05-26')) or (app_id = 20600007975571 and date in ('2018-06-16', '2018-12-08', '2018-11-17', '2018-10-27', '2018-05-19', '2018-10-20', '2018-09-29', '2018-09-22', '2018-09-01', '2018-08-25', '2018-08-04', '2018-07-28', '2018-07-07', '2018-12-29', '2018-06-30', '2018-06-09', '2018-12-22', '2018-12-01', '2018-11-10', '2018-06-02', '2018-11-03', '2018-10-13', '2018-10-06', '2018-09-15', '2018-09-08', '2018-08-18', '2018-08-11', '2018-07-21', '2018-07-14', '2018-06-23', '2019-01-05', '2018-12-15', '2018-11-24', '2018-05-26')) or (app_id = 929775122 and date in ('2017-05-06', '2017-04-15', '2018-12-08', '2018-11-17', '2018-10-27', '2017-10-28', '2017-10-07', '2017-09-16', '2018-03-31', '2018-03-10', '2017-03-11', '2017-02-18', '2017-01-28', '2018-09-22', '2018-09-01', '2017-08-12', '2017-07-22', '2018-02-03', '2018-01-13', '2018-07-28', '2018-07-07', '2017-06-17', '2017-05-27', '2018-12-29', '2017-12-09', '2017-11-18', '2018-06-02', '2018-05-12', '2018-04-21', '2017-04-22', '2017-04-01', '2018-11-03', '2018-10-13', '2017-10-14', '2017-09-23', '2017-09-02', '2018-03-17', '2018-02-24', '2017-02-04', '2017-01-14', '2018-09-08', '2018-08-18', '2017-07-29', '2017-07-08', '2018-01-20', '2017-12-30', '2018-07-14', '2018-06-23', '2017-06-03', '2017-05-13', '2019-01-05', '2018-12-15', '2018-11-24', '2017-11-25', '2017-11-04', '2018-05-19', '2018-04-28', '2018-04-07', '2017-04-08', '2017-03-18', '2017-02-25', '2018-10-20', '2018-09-29', '2017-09-30', '2017-09-09', '2017-08-19', '2018-03-03', '2018-02-10', '2017-01-21', '2018-08-25', '2018-08-04', '2017-07-15', '2017-06-24', '2018-01-06', '2017-12-16', '2018-06-30', '2018-06-09', '2017-05-20', '2017-04-29', '2018-12-22', '2018-12-01', '2018-11-10', '2017-11-11', '2017-10-21', '2018-05-05', '2018-04-14', '2018-03-24', '2017-03-25', '2017-03-04', '2017-02-11', '2018-10-06', '2018-09-15', '2017-08-26', '2017-08-05', '2018-02-17', '2018-01-27', '2017-01-07', '2018-08-11', '2018-07-21', '2017-07-01', '2017-06-10', '2017-12-23', '2017-12-02', '2018-06-16', '2018-05-26')) or (app_id = 1001424537 and date in ('2017-05-06', '2017-04-15', '2018-12-08', '2018-11-17', '2018-10-27', '2017-10-28', '2017-10-07', '2017-09-16', '2018-03-31', '2018-03-10', '2017-03-11', '2017-02-18', '2017-01-28', '2018-09-22', '2018-09-01', '2017-08-12', '2017-07-22', '2018-02-03', '2018-01-13', '2018-07-28', '2018-07-07', '2017-06-17', '2017-05-27', '2018-12-29', '2017-12-09', '2017-11-18', '2018-06-02', '2018-05-12', '2018-04-21', '2017-04-22', '2017-04-01', '2018-11-03', '2018-10-13', '2017-10-14', '2017-09-23', '2017-09-02', '2018-03-17', '2018-02-24', '2017-02-04', '2017-01-14', '2018-09-08', '2018-08-18', '2017-07-29', '2017-07-08', '2018-01-20', '2017-12-30', '2018-07-14', '2018-06-23', '2017-06-03', '2017-05-13', '2019-01-05', '2018-12-15', '2018-11-24', '2017-11-25', '2017-11-04', '2018-05-19', '2018-04-28', '2018-04-07', '2017-04-08', '2017-03-18', '2017-02-25', '2018-10-20', '2018-09-29', '2017-09-30', '2017-09-09', '2017-08-19', '2018-03-03', '2018-02-10', '2017-01-21', '2018-08-25', '2018-08-04', '2017-07-15', '2017-06-24', '2018-01-06', '2017-12-16', '2018-06-30', '2018-06-09', '2017-05-20', '2017-04-29', '2018-12-22', '2018-12-01', '2018-11-10', '2017-11-11', '2017-10-21', '2018-05-05', '2018-04-14', '2018-03-24', '2017-03-25', '2017-03-04', '2017-02-11', '2018-10-06', '2018-09-15', '2017-08-26', '2017-08-05', '2018-02-17', '2018-01-27', '2017-01-07', '2018-08-11', '2018-07-21', '2017-07-01', '2017-06-10', '2017-12-23', '2017-12-02', '2018-06-16', '2018-05-26')) or (app_id = 1037771559 and date in ('2017-04-15', '2017-04-22', '2017-04-01', '2017-04-08', '2017-03-18', '2017-02-25', '2017-03-04', '2017-02-11', '2017-03-11', '2017-02-18', '2017-01-28', '2017-02-04', '2017-01-14', '2017-01-21', '2017-05-27', '2017-05-13', '2017-05-20', '2017-04-29')) or (app_id = 1269462185 and date in ('2018-12-08', '2018-11-17', '2018-10-27', '2017-10-28', '2017-10-07', '2017-09-16', '2018-03-31', '2018-03-10', '2018-09-22', '2018-09-01', '2018-02-03', '2018-01-13', '2018-07-28', '2018-07-07', '2018-12-29', '2017-12-09', '2017-11-18', '2018-06-02', '2018-05-12', '2018-04-21', '2018-11-03', '2018-10-13', '2017-10-14', '2017-09-23', '2017-09-02', '2018-03-17', '2018-02-24', '2018-09-08', '2018-08-18', '2018-01-20', '2017-12-30', '2018-07-14', '2018-06-23', '2019-01-05', '2018-12-15', '2018-11-24', '2017-11-25', '2017-11-04', '2018-05-19', '2018-04-28', '2018-04-07', '2018-10-20', '2018-09-29', '2017-09-30', '2017-09-09', '2018-03-03', '2018-02-10', '2018-08-25', '2018-08-04', '2018-01-06', '2017-12-16', '2018-06-30', '2018-06-09', '2018-12-22', '2018-12-01', '2018-11-10', '2017-11-11', '2017-10-21', '2018-05-05', '2018-04-14', '2018-03-24', '2018-10-06', '2018-09-15', '2018-02-17', '2018-01-27', '2018-08-11', '2018-07-21', '2017-12-23', '2017-12-02', '2018-06-16', '2018-05-26')) or (app_id = 555861937 and date in ('2017-07-29', '2017-04-22', '2017-10-28', '2017-05-27', '2017-09-30', '2017-06-03', '2017-08-05', '2017-05-20')) or (app_id = 899560813 and date in ('2017-04-15', '2018-12-08', '2018-11-17', '2018-10-27', '2017-10-28', '2017-10-07', '2018-03-31', '2018-03-10', '2017-03-11', '2017-01-28', '2018-09-22', '2018-09-01', '2017-08-12', '2018-07-28', '2018-07-07', '2017-06-17', '2017-05-27', '2018-12-29', '2017-12-09', '2018-06-02', '2018-05-12', '2018-04-21', '2017-04-22', '2018-11-03', '2018-10-13', '2017-09-23', '2018-03-17', '2018-02-24', '2017-02-04', '2018-08-18', '2017-07-29', '2017-12-30', '2018-07-14', '2018-06-23', '2017-05-13', '2018-12-15', '2018-11-24', '2017-11-25', '2018-05-19', '2018-04-28', '2018-04-07', '2017-02-25', '2018-10-20', '2018-09-29', '2017-09-30', '2018-03-03', '2017-01-21', '2018-08-25', '2018-08-04', '2017-12-16', '2018-06-30', '2018-06-09', '2018-12-22', '2018-12-01', '2018-11-10', '2017-10-21', '2018-05-05', '2018-04-14', '2018-03-24', '2017-03-25', '2017-03-04', '2017-02-11', '2018-10-06', '2018-09-15', '2017-08-26', '2017-01-07', '2018-08-11', '2017-06-10', '2017-12-23', '2017-12-02', '2018-06-16', '2018-05-26')) or (app_id = 1220373112 and date in ('2018-12-08', '2018-11-17', '2018-10-27', '2017-10-28', '2017-10-07', '2017-09-16', '2018-03-31', '2018-03-10', '2018-09-22', '2018-09-01', '2017-08-12', '2017-07-22', '2018-02-03', '2018-01-13', '2018-07-28', '2018-07-07', '2017-06-17', '2017-05-27', '2018-12-29', '2017-12-09', '2017-11-18', '2018-06-02', '2018-05-12', '2018-04-21', '2018-11-03', '2018-10-13', '2017-10-14', '2017-09-23', '2017-09-02', '2018-03-17', '2018-02-24', '2018-09-08', '2018-08-18', '2017-07-29', '2017-07-08', '2018-01-20', '2017-12-30', '2018-07-14', '2018-06-23', '2017-06-03', '2019-01-05', '2018-12-15', '2018-11-24', '2017-11-25', '2017-11-04', '2018-05-19', '2018-04-28', '2018-04-07', '2018-10-20', '2018-09-29', '2017-09-30', '2017-09-09', '2017-08-19', '2018-03-03', '2018-02-10', '2018-08-25', '2018-08-04', '2017-07-15', '2017-06-24', '2018-01-06', '2017-12-16', '2018-06-30', '2018-06-09', '2018-12-22', '2018-12-01', '2018-11-10', '2017-11-11', '2017-10-21', '2018-05-05', '2018-04-14', '2018-03-24', '2018-10-06', '2018-09-15', '2017-08-26', '2017-08-05', '2018-02-17', '2018-01-27', '2018-08-11', '2018-07-21', '2017-07-01', '2017-06-10', '2017-12-23', '2017-12-02', '2018-06-16', '2018-05-26')) or (app_id = 1256271892 and date in ('2017-11-25', '2017-11-04', '2018-05-19', '2018-04-28', '2018-04-07', '2017-10-28', '2017-10-07', '2017-09-16', '2018-03-31', '2018-03-10', '2017-09-09', '2017-09-30', '2018-03-03', '2018-02-10', '2018-02-03', '2018-01-13', '2018-01-06', '2017-12-16', '2018-06-30', '2018-06-09', '2017-12-09', '2017-11-18', '2018-06-02', '2018-05-12', '2017-11-11', '2018-04-21', '2017-10-21', '2018-05-05', '2018-04-14', '2017-10-14', '2017-09-23', '2018-03-24', '2018-03-17', '2018-02-24', '2018-02-17', '2018-01-27', '2018-01-20', '2017-12-30', '2018-06-23', '2017-12-23', '2017-12-02', '2018-06-16', '2018-05-26')) or (app_id = 1209108796 and date in ('2017-11-25', '2017-11-04', '2017-05-06', '2017-10-28', '2017-10-07', '2017-09-16', '2017-09-30', '2017-09-09', '2017-08-12', '2017-06-24', '2017-12-16', '2017-12-09', '2017-11-18', '2017-05-20', '2017-04-29', '2017-11-11', '2017-10-21', '2017-04-22', '2017-10-14', '2017-09-23', '2017-09-02', '2017-08-26', '2017-07-29', '2017-12-30', '2017-07-01', '2017-06-10', '2017-12-02', '2017-06-03', '2017-05-13')) or (app_id = 20600010613334 and date in ('2018-12-22', '2018-12-29', '2019-01-05', '2018-12-15')) or (app_id = 557882383 and date in ('2017-05-06', '2017-04-15', '2018-12-08', '2018-11-17', '2018-10-27', '2017-10-28', '2017-10-07', '2017-09-16', '2018-03-31', '2018-03-10', '2017-03-11', '2017-02-18', '2017-01-28', '2018-09-22', '2018-09-01', '2017-08-12', '2017-07-22', '2018-02-03', '2018-01-13', '2018-07-28', '2018-07-07', '2017-06-17', '2017-05-27', '2018-12-29', '2017-12-09', '2017-11-18', '2018-06-02', '2018-05-12', '2018-04-21', '2017-04-22', '2017-04-01', '2018-11-03', '2018-10-13', '2017-10-14', '2017-09-23', '2017-09-02', '2018-03-17', '2018-02-24', '2017-02-04', '2017-01-14', '2018-09-08', '2018-08-18', '2017-07-29', '2017-07-08', '2018-01-20', '2017-12-30', '2018-07-14', '2018-06-23', '2017-06-03', '2017-05-13', '2019-01-05', '2018-12-15', '2018-11-24', '2017-11-25', '2017-11-04', '2018-05-19', '2018-04-28', '2018-04-07', '2017-04-08', '2017-03-18', '2017-02-25', '2018-10-20', '2018-09-29', '2017-09-30', '2017-09-09', '2017-08-19', '2018-03-03', '2018-02-10', '2017-01-21', '2018-08-25', '2018-08-04', '2017-07-15', '2017-06-24', '2018-01-06', '2017-12-16', '2018-06-30', '2018-06-09', '2017-05-20', '2017-04-29', '2018-12-22', '2018-12-01', '2018-11-10', '2017-11-11', '2017-10-21', '2018-05-05', '2018-04-14', '2018-03-24', '2017-03-25', '2017-03-04', '2017-02-11', '2018-10-06', '2018-09-15', '2017-08-26', '2017-08-05', '2018-02-17', '2018-01-27', '2017-01-07', '2018-08-11', '2018-07-21', '2017-07-01', '2017-06-10', '2017-12-23', '2017-12-02', '2018-06-16', '2018-05-26')) or (app_id = 1329681380 and date in ('2018-12-22', '2018-12-01', '2018-12-08', '2018-12-29', '2019-01-05', '2018-12-15', '2018-11-24')) or (app_id = 20600006389088 and date in ('2018-06-16', '2018-12-08', '2018-11-17', '2018-05-19', '2018-04-28', '2018-04-07', '2018-10-27', '2018-10-20', '2018-09-29', '2018-03-31', '2018-09-22', '2018-09-01', '2018-08-25', '2018-08-04', '2018-07-28', '2018-07-07', '2018-12-29', '2018-06-30', '2018-06-09', '2018-12-22', '2018-12-01', '2018-11-10', '2018-05-12', '2018-04-21', '2018-06-02', '2018-11-03', '2018-10-13', '2018-04-14', '2018-05-05', '2018-03-24', '2018-10-06', '2018-09-15', '2018-09-08', '2018-08-18', '2018-08-11', '2018-07-21', '2018-07-14', '2018-06-23', '2019-01-05', '2018-12-15', '2018-11-24', '2018-05-26')) or (app_id = 20600004951990 and date in ('2017-04-01', '2017-04-08', '2017-03-18', '2017-02-25', '2017-03-25', '2017-03-04', '2017-02-11', '2017-03-11', '2017-02-18', '2017-01-28', '2017-02-04', '2017-01-14', '2017-01-21', '2017-01-07')) or (app_id = 20600010141346 and date in ('2018-08-18', '2018-12-22', '2018-11-10', '2018-12-01', '2018-08-04', '2018-11-17', '2018-08-11', '2018-07-21', '2018-07-28', '2018-09-29', '2018-12-29', '2018-09-22', '2018-11-24')) or (app_id = 946826828 and date in ('2017-01-21', '2017-01-14')) or (app_id = 20600005563296 and date in ('2018-01-13', '2018-01-06', '2018-06-23', '2018-06-16')) or (app_id = 20600009788956 and date in ('2018-06-16', '2018-12-08', '2018-11-17', '2018-10-27', '2018-05-19', '2018-10-20', '2018-09-29', '2018-09-22', '2018-09-01', '2018-08-25', '2018-08-04', '2018-07-28', '2018-07-07', '2018-12-29', '2018-06-30', '2018-06-09', '2018-12-22', '2018-12-01', '2018-06-02', '2018-11-10', '2018-11-03', '2018-10-13', '2018-10-06', '2018-09-15', '2018-09-08', '2018-08-18', '2018-08-11', '2018-07-21', '2018-07-14', '2018-06-23', '2019-01-05', '2018-12-15', '2018-11-24', '2018-05-26')) or (app_id = 909585896 and date in ('2017-05-06', '2017-04-15', '2018-12-08', '2018-11-17', '2018-10-27', '2017-10-28', '2017-10-07', '2017-09-16', '2018-03-31', '2018-03-10', '2017-03-11', '2017-02-18', '2017-01-28', '2018-09-22', '2018-09-01', '2017-08-12', '2017-07-22', '2018-02-03', '2018-01-13', '2018-07-28', '2018-07-07', '2017-06-17', '2017-05-27', '2018-12-29', '2017-12-09', '2017-11-18', '2018-06-02', '2018-05-12', '2018-04-21', '2017-04-22', '2017-04-01', '2018-11-03', '2018-10-13', '2017-10-14', '2017-09-23', '2017-09-02', '2018-03-17', '2018-02-24', '2017-02-04', '2017-01-14', '2018-09-08', '2018-08-18', '2017-07-29', '2017-07-08', '2018-01-20', '2017-12-30', '2018-07-14', '2018-06-23', '2017-06-03', '2017-05-13', '2019-01-05', '2018-12-15', '2018-11-24', '2017-11-25', '2017-11-04', '2018-05-19', '2018-04-28', '2018-04-07', '2017-04-08', '2017-03-18', '2017-02-25', '2018-10-20', '2018-09-29', '2017-09-30', '2017-09-09', '2017-08-19', '2018-03-03', '2018-02-10', '2017-01-21', '2018-08-25', '2018-08-04', '2017-07-15', '2017-06-24', '2018-01-06', '2017-12-16', '2018-06-30', '2018-06-09', '2017-05-20', '2017-04-29', '2018-12-22', '2018-12-01', '2018-11-10', '2017-11-11', '2017-10-21', '2018-05-05', '2018-04-14', '2018-03-24', '2017-03-25', '2017-03-04', '2017-02-11', '2018-10-06', '2018-09-15', '2017-08-26', '2017-08-05', '2018-02-17', '2018-01-27', '2017-01-07', '2018-08-11', '2018-07-21', '2017-07-01', '2017-06-10', '2017-12-23', '2017-12-02', '2018-06-16', '2018-05-26')) or (app_id = 20600000578785 and date in ('2018-06-16', '2018-12-08', '2018-11-17', '2018-10-27', '2018-04-28', '2018-05-19', '2018-04-07', '2018-10-20', '2018-09-29', '2018-03-31', '2018-03-10', '2018-09-22', '2018-09-01', '2018-03-03', '2018-02-10', '2018-08-25', '2018-08-04', '2018-02-03', '2018-01-13', '2018-07-28', '2018-07-07', '2018-01-06', '2018-12-29', '2018-06-30', '2018-06-09', '2018-12-22', '2018-12-01', '2018-06-02', '2018-05-12', '2018-04-21', '2018-11-10', '2018-11-03', '2018-10-13', '2018-04-14', '2018-03-24', '2018-05-05', '2018-10-06', '2018-09-15', '2018-03-17', '2018-02-24', '2018-09-08', '2018-08-18', '2018-02-17', '2018-01-27', '2018-08-11', '2018-07-21', '2018-01-20', '2017-12-30', '2018-07-14', '2018-06-23', '2019-01-05', '2018-12-15', '2018-11-24', '2018-05-26')) or (app_id = 20600010842893 and date in ('2018-12-22')) or (app_id = 555272782 and date in ('2017-04-22', '2018-03-03')) or (app_id = 972826307 and date in ('2017-05-06', '2017-04-15', '2018-12-08', '2018-11-17', '2018-10-27', '2017-10-28', '2017-10-07', '2017-09-16', '2018-03-31', '2018-03-10', '2017-03-11', '2017-02-18', '2017-01-28', '2018-09-22', '2018-09-01', '2017-08-12', '2017-07-22', '2018-02-03', '2018-01-13', '2018-07-28', '2018-07-07', '2017-06-17', '2017-05-27', '2018-12-29', '2017-12-09', '2017-11-18', '2018-06-02', '2018-05-12', '2018-04-21', '2017-04-22', '2017-04-01', '2018-11-03', '2018-10-13', '2017-10-14', '2017-09-23', '2017-09-02', '2018-03-17', '2018-02-24', '2017-02-04', '2017-01-14', '2018-09-08', '2018-08-18', '2017-07-29', '2017-07-08', '2018-01-20', '2017-12-30', '2018-07-14', '2018-06-23', '2017-06-03', '2017-05-13', '2019-01-05', '2018-12-15', '2018-11-24', '2017-11-25', '2017-11-04', '2018-05-19', '2018-04-28', '2018-04-07', '2017-04-08', '2017-03-18', '2017-02-25', '2018-10-20', '2018-09-29', '2017-09-30', '2017-09-09', '2017-08-19', '2018-03-03', '2018-02-10', '2017-01-21', '2018-08-25', '2018-08-04', '2017-07-15', '2017-06-24', '2018-01-06', '2017-12-16', '2018-06-30', '2018-06-09', '2017-05-20', '2017-04-29', '2018-12-22', '2018-12-01', '2018-11-10', '2017-11-11', '2017-10-21', '2018-05-05', '2018-04-14', '2018-03-24', '2017-03-25', '2017-03-04', '2017-02-11', '2018-10-06', '2018-09-15', '2017-08-26', '2017-08-05', '2018-02-17', '2018-01-27', '2017-01-07', '2018-08-11', '2018-07-21', '2017-07-01', '2017-06-10', '2017-12-23', '2017-12-02', '2018-06-16', '2018-05-26')) or (app_id = 20600004428807 and date in ('2017-05-06', '2017-04-15', '2018-12-08', '2018-11-17', '2018-10-27', '2017-10-28', '2017-10-07', '2017-09-16', '2018-03-31', '2018-03-10', '2017-03-11', '2017-02-18', '2017-01-28', '2018-09-22', '2018-09-01', '2017-08-12', '2017-07-22', '2018-02-03', '2018-01-13', '2018-07-28', '2018-07-07', '2017-06-17', '2017-05-27', '2018-12-29', '2017-12-09', '2017-11-18', '2018-06-02', '2018-05-12', '2018-04-21', '2017-04-22', '2017-04-01', '2018-11-03', '2018-10-13', '2017-10-14', '2017-09-23', '2017-09-02', '2018-03-17', '2018-02-24', '2017-02-04', '2017-01-14', '2018-09-08', '2018-08-18', '2017-07-29', '2017-07-08', '2018-01-20', '2017-12-30', '2018-07-14', '2018-06-23', '2017-06-03', '2017-05-13', '2019-01-05', '2018-12-15', '2018-11-24', '2017-11-25', '2017-11-04', '2018-05-19', '2018-04-28', '2018-04-07', '2017-04-08', '2017-03-18', '2017-02-25', '2018-10-20', '2018-09-29', '2017-09-30', '2017-09-09', '2017-08-19', '2018-03-03', '2018-02-10', '2017-01-21', '2018-08-25', '2018-08-04', '2017-07-15', '2017-06-24', '2018-01-06', '2017-12-16', '2018-06-30', '2018-06-09', '2017-05-20', '2017-04-29', '2018-12-22', '2018-12-01', '2018-11-10', '2017-11-11', '2017-10-21', '2018-05-05', '2018-04-14', '2018-03-24', '2017-03-25', '2017-03-04', '2017-02-11', '2018-10-06', '2018-09-15', '2017-08-26', '2017-08-05', '2018-02-17', '2018-01-27', '2017-01-07', '2018-08-11', '2018-07-21', '2017-07-01', '2017-06-10', '2017-12-23', '2017-12-02', '2018-06-16', '2018-05-26')) or (app_id = 646772168 and date in ('2017-05-06', '2017-04-15', '2017-10-28', '2017-10-07', '2017-09-16', '2017-03-11', '2017-02-18', '2018-09-22', '2018-09-01', '2018-02-03', '2018-07-28', '2018-07-07', '2017-06-17', '2017-05-27', '2017-12-09', '2017-11-18', '2018-06-02', '2017-04-22', '2017-10-14', '2017-09-23', '2017-09-02', '2018-03-17', '2017-01-14', '2018-08-18', '2017-07-29', '2018-01-20', '2017-12-30', '2018-06-23', '2017-06-03', '2017-05-13', '2019-01-05', '2018-11-24', '2017-11-25', '2018-04-28', '2017-04-08', '2017-03-18', '2017-02-25', '2018-10-20', '2017-09-30', '2017-09-09', '2018-08-25', '2017-07-15', '2017-06-24', '2017-12-16', '2018-06-09', '2017-05-20', '2017-04-29', '2017-11-11', '2018-05-05', '2018-04-14', '2017-03-25', '2017-03-04', '2017-08-26', '2017-08-05', '2018-02-17', '2018-01-27', '2017-01-07', '2018-08-11', '2018-07-21', '2017-06-10', '2017-12-02', '2018-06-16', '2018-05-26')) or (app_id = 1164066657 and date in ('2017-05-06', '2017-04-15', '2018-11-17', '2018-10-27', '2017-10-28', '2017-10-07', '2017-09-16', '2018-03-31', '2018-03-10', '2017-03-11', '2017-02-18', '2017-01-28', '2018-09-22', '2018-09-01', '2017-08-12', '2017-07-22', '2018-02-03', '2018-01-13', '2018-07-28', '2018-07-07', '2017-06-17', '2017-05-27', '2017-12-09', '2017-11-18', '2018-06-02', '2018-05-12', '2018-04-21', '2017-04-22', '2017-04-01', '2018-11-03', '2018-10-13', '2017-10-14', '2017-09-23', '2017-09-02', '2018-03-17', '2018-02-24', '2017-02-04', '2017-01-14', '2018-09-08', '2018-08-18', '2017-07-29', '2017-07-08', '2018-01-20', '2017-12-30', '2018-07-14', '2018-06-23', '2017-06-03', '2017-05-13', '2018-11-24', '2017-11-25', '2017-11-04', '2018-05-19', '2018-04-28', '2018-04-07', '2017-04-08', '2017-03-18', '2017-02-25', '2018-10-20', '2018-09-29', '2017-09-30', '2017-09-09', '2017-08-19', '2018-03-03', '2018-02-10', '2017-01-21', '2018-08-25', '2018-08-04', '2017-07-15', '2017-06-24', '2018-01-06', '2017-12-16', '2018-06-30', '2018-06-09', '2017-05-20', '2017-04-29', '2018-12-01', '2018-11-10', '2017-11-11', '2017-10-21', '2018-05-05', '2018-04-14', '2018-03-24', '2017-03-25', '2017-03-04', '2017-02-11', '2018-10-06', '2018-09-15', '2017-08-26', '2017-08-05', '2018-02-17', '2018-01-27', '2017-01-07', '2018-08-11', '2018-07-21', '2017-07-01', '2017-06-10', '2017-12-23', '2017-12-02', '2018-06-16', '2018-05-26')) or (app_id = 20600007651949 and date in ('2018-12-08', '2018-11-17', '2018-10-27', '2017-10-28', '2017-10-07', '2017-09-16', '2018-03-31', '2018-03-10', '2018-09-22', '2018-09-01', '2017-08-12', '2017-07-22', '2018-02-03', '2018-01-13', '2018-07-28', '2018-07-07', '2018-12-29', '2017-12-09', '2017-11-18', '2018-06-02', '2018-05-12', '2018-04-21', '2018-11-03', '2018-10-13', '2017-10-14', '2017-09-23', '2017-09-02', '2018-03-17', '2018-02-24', '2018-09-08', '2018-08-18', '2017-07-29', '2017-07-08', '2018-01-20', '2017-12-30', '2018-07-14', '2018-06-23', '2019-01-05', '2018-12-15', '2018-11-24', '2017-11-25', '2017-11-04', '2018-05-19', '2018-04-28', '2018-04-07', '2018-10-20', '2018-09-29', '2017-09-30', '2017-09-09', '2017-08-19', '2018-03-03', '2018-02-10', '2018-08-25', '2018-08-04', '2017-07-15', '2017-06-24', '2018-01-06', '2017-12-16', '2018-06-30', '2018-06-09', '2018-12-22', '2018-12-01', '2018-11-10', '2017-11-11', '2017-10-21', '2018-05-05', '2018-04-14', '2018-03-24', '2018-10-06', '2018-09-15', '2017-08-26', '2017-08-05', '2018-02-17', '2018-01-27', '2018-08-11', '2018-07-21', '2017-07-01', '2017-12-23', '2017-12-02', '2018-06-16', '2018-05-26')) or (app_id = 1300096411 and date in ('2018-05-26', '2018-12-08', '2018-11-17', '2018-10-27', '2018-04-28', '2018-04-07', '2018-05-19', '2018-10-20', '2018-09-29', '2018-03-31', '2018-03-10', '2018-09-22', '2018-09-01', '2018-03-03', '2018-02-10', '2018-08-25', '2018-08-04', '2018-02-03', '2018-07-28', '2018-07-07', '2018-12-29', '2018-06-30', '2018-06-09', '2018-12-22', '2018-12-01', '2018-06-02', '2018-05-12', '2018-04-21', '2018-11-10', '2018-11-03', '2018-10-13', '2018-04-14', '2018-03-24', '2018-05-05', '2018-10-06', '2018-09-15', '2018-03-17', '2018-02-24', '2018-09-08', '2018-08-18', '2018-02-17', '2018-01-27', '2018-08-11', '2018-07-21', '2018-07-14', '2018-06-23', '2019-01-05', '2018-12-15', '2018-11-24', '2018-06-16')) or (app_id = 1378856776 and date in ('2018-12-22', '2018-12-29', '2019-01-05', '2018-12-15')) or (app_id = 1300048349 and date in ('2018-12-22', '2018-12-01', '2018-12-08', '2018-04-28', '2018-11-03', '2018-05-05', '2018-10-13', '2018-09-29', '2018-10-06', '2018-09-22', '2018-09-01', '2018-09-08', '2018-08-04', '2018-07-28', '2018-07-07', '2018-06-23', '2018-12-29', '2018-06-30')) or (app_id = 20600003231650 and date in ('2017-03-18', '2017-07-01', '2017-08-12')) or (app_id = 20600006416981 and date in ('2017-01-21', '2017-02-11', '2017-02-04', '2017-01-14')) or (app_id = 993179311 and date in ('2017-04-15', '2018-07-07', '2018-06-16')) or (app_id = 1418529000 and date in ('2018-09-08', '2018-08-18', '2018-08-25', '2018-10-13', '2018-10-20', '2018-09-29', '2018-10-06', '2018-09-15', '2018-09-22', '2018-09-01')) or (app_id = 20600005725213 and date in ('2017-05-06', '2018-12-08', '2018-11-17', '2018-10-27', '2017-10-28', '2017-10-07', '2017-09-16', '2018-03-31', '2018-03-10', '2018-09-22', '2018-09-01', '2017-08-12', '2017-07-22', '2018-02-03', '2018-01-13', '2018-07-28', '2018-07-07', '2017-06-17', '2017-05-27', '2018-12-29', '2017-12-09', '2017-11-18', '2018-06-02', '2018-05-12', '2018-04-21', '2018-11-03', '2018-10-13', '2017-10-14', '2017-09-23', '2017-09-02', '2018-03-17', '2018-02-24', '2018-09-08', '2018-08-18', '2017-07-29', '2017-07-08', '2018-01-20', '2017-12-30', '2018-07-14', '2018-06-23', '2017-06-03', '2017-05-13', '2019-01-05', '2018-12-15', '2018-11-24', '2017-11-25', '2017-11-04', '2018-05-19', '2018-04-28', '2018-04-07', '2018-10-20', '2018-09-29', '2017-09-30', '2017-09-09', '2017-08-19', '2018-03-03', '2018-02-10', '2018-08-25', '2018-08-04', '2017-07-15', '2017-06-24', '2018-01-06', '2017-12-16', '2018-06-30', '2018-06-09', '2017-05-20', '2017-04-29', '2018-12-22', '2018-12-01', '2018-11-10', '2017-11-11', '2017-10-21', '2018-05-05', '2018-04-14', '2018-03-24', '2018-10-06', '2018-09-15', '2017-08-26', '2017-08-05', '2018-02-17', '2018-01-27', '2018-08-11', '2018-07-21', '2017-07-01', '2017-06-10', '2017-12-23', '2017-12-02', '2018-06-16', '2018-05-26')) or (app_id = 560901396 and date in ('2017-05-06', '2017-04-15', '2018-12-08', '2018-11-17', '2018-10-27', '2017-10-28', '2017-10-07', '2017-09-16', '2018-03-31', '2018-03-10', '2017-03-11', '2017-02-18', '2017-01-28', '2018-09-22', '2018-09-01', '2017-08-12', '2017-07-22', '2018-02-03', '2018-01-13', '2018-07-28', '2018-07-07', '2017-06-17', '2017-05-27', '2018-12-29', '2017-12-09', '2017-11-18', '2018-06-02', '2018-05-12', '2018-04-21', '2017-04-22', '2017-04-01', '2018-11-03', '2018-10-13', '2017-10-14', '2017-09-23', '2017-09-02', '2018-03-17', '2018-02-24', '2017-02-04', '2017-01-14', '2018-09-08', '2018-08-18', '2017-07-29', '2017-07-08', '2018-01-20', '2017-12-30', '2018-07-14', '2018-06-23', '2017-06-03', '2017-05-13', '2019-01-05', '2018-12-15', '2018-11-24', '2017-11-25', '2017-11-04', '2018-05-19', '2018-04-28', '2018-04-07', '2017-04-08', '2017-03-18', '2017-02-25', '2018-10-20', '2018-09-29', '2017-09-30', '2017-09-09', '2017-08-19', '2018-03-03', '2018-02-10', '2017-01-21', '2018-08-25', '2018-08-04', '2017-07-15', '2017-06-24', '2018-01-06', '2017-12-16', '2018-06-30', '2018-06-09', '2017-05-20', '2017-04-29', '2018-12-22', '2018-12-01', '2018-11-10', '2017-11-11', '2017-10-21', '2018-05-05', '2018-04-14', '2018-03-24', '2017-03-25', '2017-03-04', '2017-02-11', '2018-10-06', '2018-09-15', '2017-08-26', '2017-08-05', '2018-02-17', '2018-01-27', '2017-01-07', '2018-08-11', '2018-07-21', '2017-07-01', '2017-06-10', '2017-12-23', '2017-12-02', '2018-06-16', '2018-05-26')) or (app_id = 1111953146 and date in ('2017-05-06', '2017-04-15', '2018-12-08', '2018-11-17', '2018-10-27', '2017-10-28', '2017-10-07', '2017-09-16', '2018-03-31', '2018-03-10', '2017-03-11', '2017-02-18', '2017-01-28', '2018-09-22', '2018-09-01', '2017-08-12', '2017-07-22', '2018-02-03', '2018-01-13', '2018-07-28', '2018-07-07', '2017-06-17', '2017-05-27', '2018-12-29', '2017-12-09', '2017-11-18', '2018-06-02', '2018-05-12', '2018-04-21', '2017-04-22', '2017-04-01', '2018-11-03', '2018-10-13', '2017-10-14', '2017-09-23', '2017-09-02', '2018-03-17', '2018-02-24', '2017-02-04', '2017-01-14', '2018-09-08', '2018-08-18', '2017-07-29', '2017-07-08', '2018-01-20', '2017-12-30', '2018-07-14', '2018-06-23', '2017-06-03', '2017-05-13', '2019-01-05', '2018-12-15', '2018-11-24', '2017-11-25', '2017-11-04', '2018-05-19', '2018-04-28', '2018-04-07', '2017-04-08', '2017-03-18', '2017-02-25', '2018-10-20', '2018-09-29', '2017-09-30', '2017-09-09', '2017-08-19', '2018-03-03', '2018-02-10', '2017-01-21', '2018-08-25', '2018-08-04', '2017-07-15', '2017-06-24', '2018-01-06', '2017-12-16', '2018-06-30', '2018-06-09', '2017-05-20', '2017-04-29', '2018-12-22', '2018-12-01', '2018-11-10', '2017-11-11', '2017-10-21', '2018-05-05', '2018-04-14', '2018-03-24', '2017-03-25', '2017-03-04', '2017-02-11', '2018-10-06', '2018-09-15', '2017-08-26', '2017-08-05', '2018-02-17', '2018-01-27', '2017-01-07', '2018-08-11', '2018-07-21', '2017-07-01', '2017-06-10', '2017-12-23', '2017-12-02', '2018-06-16', '2018-05-26')) or (app_id = 995661933 and date in ('2017-05-06', '2017-04-15', '2018-12-08', '2018-11-17', '2018-10-27', '2017-10-28', '2017-10-07', '2017-09-16', '2018-03-31', '2018-03-10', '2017-03-11', '2017-02-18', '2017-01-28', '2018-09-22', '2018-09-01', '2017-08-12', '2017-07-22', '2018-02-03', '2018-01-13', '2018-07-28', '2018-07-07', '2017-06-17', '2017-05-27', '2018-12-29', '2017-12-09', '2017-11-18', '2018-06-02', '2018-05-12', '2018-04-21', '2017-04-22', '2017-04-01', '2018-11-03', '2018-10-13', '2017-10-14', '2017-09-23', '2017-09-02', '2018-03-17', '2018-02-24', '2017-02-04', '2017-01-14', '2018-09-08', '2018-08-18', '2017-07-29', '2017-07-08', '2018-01-20', '2017-12-30', '2018-07-14', '2018-06-23', '2017-06-03', '2017-05-13', '2019-01-05', '2018-12-15', '2018-11-24', '2017-11-25', '2017-11-04', '2018-05-19', '2018-04-28', '2018-04-07', '2017-04-08', '2017-03-18', '2017-02-25', '2018-10-20', '2018-09-29', '2017-09-30', '2017-09-09', '2017-08-19', '2018-03-03', '2018-02-10', '2017-01-21', '2018-08-25', '2018-08-04', '2017-07-15', '2017-06-24', '2018-01-06', '2017-12-16', '2018-06-30', '2018-06-09', '2017-05-20', '2017-04-29', '2018-12-22', '2018-12-01', '2018-11-10', '2017-11-11', '2017-10-21', '2018-05-05', '2018-04-14', '2018-03-24', '2017-03-25', '2017-03-04', '2017-02-11', '2018-10-06', '2018-09-15', '2017-08-26', '2017-08-05', '2018-02-17', '2018-01-27', '2017-01-07', '2018-08-11', '2018-07-21', '2017-07-01', '2017-06-10', '2017-12-23', '2017-12-02', '2018-06-16', '2018-05-26')) or (app_id = 20600011048689 and date in ('2018-12-22', '2018-12-29', '2019-01-05', '2018-12-15')) or (app_id = 20600003491447 and date in ('2017-05-06', '2017-04-15', '2018-12-08', '2018-11-17', '2018-10-27', '2017-10-28', '2017-10-07', '2017-09-16', '2018-03-31', '2018-03-10', '2017-03-11', '2017-02-18', '2017-01-28', '2018-09-22', '2018-09-01', '2017-08-12', '2017-07-22', '2018-02-03', '2018-01-13', '2018-07-28', '2018-07-07', '2017-06-17', '2017-05-27', '2018-12-29', '2017-12-09', '2017-11-18', '2018-06-02', '2018-05-12', '2018-04-21', '2017-04-22', '2017-04-01', '2018-11-03', '2018-10-13', '2017-10-14', '2017-09-23', '2017-09-02', '2018-03-17', '2018-02-24', '2017-02-04', '2017-01-14', '2018-09-08', '2018-08-18', '2017-07-29', '2017-07-08', '2018-01-20', '2017-12-30', '2018-07-14', '2018-06-23', '2017-06-03', '2017-05-13', '2019-01-05', '2018-12-15', '2018-11-24', '2017-11-25', '2017-11-04', '2018-05-19', '2018-04-28', '2018-04-07', '2017-04-08', '2017-03-18', '2017-02-25', '2018-10-20', '2018-09-29', '2017-09-30', '2017-09-09', '2017-08-19', '2018-03-03', '2018-02-10', '2017-01-21', '2018-08-25', '2018-08-04', '2017-07-15', '2017-06-24', '2018-01-06', '2017-12-16', '2018-06-30', '2018-06-09', '2017-05-20', '2017-04-29', '2018-12-22', '2018-12-01', '2018-11-10', '2017-11-11', '2017-10-21', '2018-05-05', '2018-04-14', '2018-03-24', '2017-03-25', '2017-03-04', '2017-02-11', '2018-10-06', '2018-09-15', '2017-08-26', '2017-08-05', '2018-02-17', '2018-01-27', '2017-01-07', '2018-08-11', '2018-07-21', '2017-07-01', '2017-06-10', '2017-12-23', '2017-12-02', '2018-06-16', '2018-05-26')) or (app_id = 796784930 and date in ('2017-09-23')) or (app_id = 1331478987 and date in ('2018-12-22', '2018-12-01', '2018-12-08', '2018-11-17', '2018-12-29', '2019-01-05', '2018-12-15', '2018-11-24')) or (app_id = 20600003782021 and date in ('2017-05-06', '2017-04-15', '2018-12-08', '2018-11-17', '2018-10-27', '2017-10-28', '2017-10-07', '2017-09-16', '2018-03-31', '2018-03-10', '2017-03-11', '2017-02-18', '2017-01-28', '2018-09-22', '2018-09-01', '2017-08-12', '2017-07-22', '2018-02-03', '2018-01-13', '2018-07-28', '2018-07-07', '2017-06-17', '2017-05-27', '2018-12-29', '2017-12-09', '2017-11-18', '2018-06-02', '2018-05-12', '2018-04-21', '2017-04-22', '2017-04-01', '2018-11-03', '2018-10-13', '2017-10-14', '2017-09-23', '2017-09-02', '2018-03-17', '2018-02-24', '2017-02-04', '2017-01-14', '2018-09-08', '2018-08-18', '2017-07-29', '2017-07-08', '2018-01-20', '2017-12-30', '2018-07-14', '2018-06-23', '2017-06-03', '2017-05-13', '2019-01-05', '2018-12-15', '2018-11-24', '2017-11-25', '2017-11-04', '2018-05-19', '2018-04-28', '2018-04-07', '2017-04-08', '2017-03-18', '2017-02-25', '2018-10-20', '2018-09-29', '2017-09-30', '2017-09-09', '2017-08-19', '2018-03-03', '2018-02-10', '2017-01-21', '2018-08-25', '2018-08-04', '2017-07-15', '2017-06-24', '2018-01-06', '2017-12-16', '2018-06-30', '2018-06-09', '2017-05-20', '2017-04-29', '2018-12-22', '2018-12-01', '2018-11-10', '2017-11-11', '2017-10-21', '2018-05-05', '2018-04-14', '2018-03-24', '2017-03-25', '2017-03-04', '2017-02-11', '2018-10-06', '2018-09-15', '2017-08-26', '2017-08-05', '2018-02-17', '2018-01-27', '2017-01-07', '2018-08-11', '2018-07-21', '2017-07-01', '2017-06-10', '2017-12-23', '2017-12-02', '2018-06-16', '2018-05-26')) or (app_id = 20600010076205 and date in ('2018-12-01', '2018-12-08', '2018-11-17', '2018-11-03', '2018-09-29', '2018-09-22', '2018-09-01', '2018-09-08', '2018-08-18', '2018-08-25', '2018-08-04', '2018-08-11', '2018-07-21', '2018-07-28', '2018-07-14', '2018-12-15', '2018-11-24')) or (app_id = 20600009805990 and date in ('2018-08-04', '2018-07-28')) or (app_id = 1274061132 and date in ('2018-11-24', '2018-12-08', '2018-11-17', '2018-10-27', '2018-05-19', '2018-04-07', '2018-04-28', '2018-10-20', '2018-09-29', '2018-03-31', '2018-03-10', '2018-09-22', '2018-09-01', '2018-03-03', '2018-02-10', '2018-08-25', '2018-08-04', '2018-02-03', '2018-01-13', '2018-07-28', '2018-07-07', '2018-01-06', '2018-12-29', '2018-06-30', '2018-06-09', '2018-12-22', '2018-12-01', '2018-11-10', '2018-06-02', '2018-04-21', '2018-05-12', '2018-11-03', '2018-05-05', '2018-10-13', '2018-04-14', '2018-03-24', '2018-10-06', '2018-09-15', '2018-03-17', '2018-02-24', '2018-09-08', '2018-08-18', '2018-02-17', '2018-01-27', '2018-08-11', '2018-07-21', '2018-01-20', '2017-12-30', '2018-07-14', '2018-06-23', '2017-12-23', '2019-01-05', '2018-12-15', '2018-06-16', '2018-05-26')) or (app_id = 1345275897 and date in ('2018-06-16', '2018-12-08', '2018-11-17', '2018-05-19', '2018-04-28', '2018-10-27', '2018-04-07', '2018-10-20', '2018-09-29', '2018-03-31', '2018-09-22', '2018-09-01', '2018-08-25', '2018-08-04', '2018-07-28', '2018-07-07', '2018-12-29', '2018-06-30', '2018-06-09', '2018-12-22', '2018-12-01', '2018-11-10', '2018-05-12', '2018-04-21', '2018-06-02', '2018-11-03', '2018-10-13', '2018-04-14', '2018-05-05', '2018-10-06', '2018-09-15', '2018-09-08', '2018-08-18', '2018-08-11', '2018-07-21', '2018-07-14', '2018-06-23', '2019-01-05', '2018-12-15', '2018-11-24', '2018-05-26')) or (app_id = 20600006376245 and date in ('2017-10-21', '2017-10-07', '2017-02-25', '2017-02-11', '2017-09-30', '2017-08-19', '2017-02-18', '2017-01-28', '2017-08-05', '2017-02-04', '2017-01-14', '2017-08-12', '2017-01-21', '2017-01-07')) or (app_id = 20600005771435 and date in ('2018-10-27', '2017-10-28', '2018-03-31', '2018-03-10', '2018-09-22', '2018-09-01', '2018-02-03', '2018-01-13', '2018-07-28', '2018-07-07', '2017-12-09', '2017-11-18', '2018-06-02', '2018-05-12', '2018-04-21', '2018-11-03', '2018-10-13', '2017-10-14', '2018-03-17', '2018-02-24', '2018-09-08', '2018-08-18', '2018-01-20', '2017-12-30', '2018-07-14', '2018-06-23', '2018-11-24', '2017-11-25', '2017-11-04', '2018-05-19', '2018-04-28', '2018-04-07', '2018-10-20', '2018-09-29', '2018-03-03', '2018-02-10', '2018-08-25', '2018-01-06', '2017-12-16', '2018-06-30', '2018-06-09', '2018-12-01', '2018-11-10', '2017-11-11', '2017-10-21', '2018-05-05', '2018-04-14', '2018-03-24', '2018-10-06', '2018-09-15', '2018-02-17', '2018-01-27', '2018-08-11', '2018-07-21', '2017-12-23', '2017-12-02', '2018-06-16', '2018-05-26')) or (app_id = 20600000695491 and date in ('2017-05-06', '2017-04-15', '2018-12-08', '2018-11-17', '2018-10-27', '2017-10-28', '2017-10-07', '2017-09-16', '2018-03-31', '2018-03-10', '2017-03-11', '2017-02-18', '2017-01-28', '2018-09-22', '2018-09-01', '2017-08-12', '2017-07-22', '2018-02-03', '2018-01-13', '2018-07-28', '2018-07-07', '2017-06-17', '2017-05-27', '2018-12-29', '2017-12-09', '2017-11-18', '2018-06-02', '2018-05-12', '2018-04-21', '2017-04-22', '2017-04-01', '2018-11-03', '2018-10-13', '2017-10-14', '2017-09-23', '2017-09-02', '2018-03-17', '2018-02-24', '2017-02-04', '2017-01-14', '2018-09-08', '2018-08-18', '2017-07-29', '2017-07-08', '2018-01-20', '2017-12-30', '2018-07-14', '2018-06-23', '2017-06-03', '2017-05-13', '2019-01-05', '2018-12-15', '2018-11-24', '2017-11-25', '2017-11-04', '2018-05-19', '2018-04-28', '2018-04-07', '2017-04-08', '2017-03-18', '2017-02-25', '2018-10-20', '2018-09-29', '2017-09-30', '2017-09-09', '2017-08-19', '2018-03-03', '2018-02-10', '2017-01-21', '2018-08-25', '2018-08-04', '2017-07-15', '2017-06-24', '2018-01-06', '2017-12-16', '2018-06-30', '2018-06-09', '2017-05-20', '2017-04-29', '2018-12-22', '2018-12-01', '2018-11-10', '2017-11-11', '2017-10-21', '2018-05-05', '2018-04-14', '2018-03-24', '2017-03-25', '2017-03-04', '2017-02-11', '2018-10-06', '2018-09-15', '2017-08-26', '2017-08-05', '2018-02-17', '2018-01-27', '2017-01-07', '2018-08-11', '2018-07-21', '2017-07-01', '2017-06-10', '2017-12-23', '2017-12-02', '2018-06-16', '2018-05-26')) or (app_id = 20600006623068 and date in ('2017-05-06', '2017-04-15', '2018-12-08', '2018-11-17', '2018-10-27', '2017-10-28', '2017-10-07', '2017-09-16', '2018-03-31', '2018-03-10', '2017-03-11', '2017-02-18', '2017-01-28', '2018-09-22', '2018-09-01', '2017-08-12', '2017-07-22', '2018-02-03', '2018-01-13', '2018-07-28', '2018-07-07', '2017-06-17', '2017-05-27', '2018-12-29', '2017-12-09', '2017-11-18', '2018-06-02', '2018-05-12', '2018-04-21', '2017-04-22', '2017-04-01', '2018-11-03', '2018-10-13', '2017-10-14', '2017-09-23', '2017-09-02', '2018-03-17', '2018-02-24', '2017-02-04', '2017-01-14', '2018-09-08', '2018-08-18', '2017-07-29', '2017-07-08', '2018-01-20', '2017-12-30', '2018-07-14', '2018-06-23', '2017-06-03', '2017-05-13', '2019-01-05', '2018-12-15', '2018-11-24', '2017-11-25', '2017-11-04', '2018-05-19', '2018-04-28', '2018-04-07', '2017-04-08', '2017-03-18', '2017-02-25', '2018-10-20', '2018-09-29', '2017-09-30', '2017-09-09', '2017-08-19', '2018-03-03', '2018-02-10', '2017-01-21', '2018-08-25', '2018-08-04', '2017-07-15', '2017-06-24', '2018-01-06', '2017-12-16', '2018-06-30', '2018-06-09', '2017-05-20', '2017-04-29', '2018-12-22', '2018-12-01', '2018-11-10', '2017-11-11', '2017-10-21', '2018-05-05', '2018-04-14', '2018-03-24', '2017-03-25', '2017-03-04', '2017-02-11', '2018-10-06', '2018-09-15', '2017-08-26', '2017-08-05', '2018-02-17', '2018-01-27', '2017-01-07', '2018-08-11', '2018-07-21', '2017-07-01', '2017-06-10', '2017-12-23', '2017-12-02', '2018-06-16', '2018-05-26')) or (app_id = 20600009796160 and date in ('2018-11-24', '2018-12-22', '2018-12-01', '2018-06-02', '2018-11-10', '2018-12-08', '2018-11-17', '2018-10-27', '2018-11-03', '2018-10-20', '2018-09-29', '2018-10-06', '2018-08-18', '2018-08-04', '2018-08-11', '2018-07-21', '2018-07-28', '2018-07-07', '2018-07-14', '2018-06-23', '2018-12-29', '2018-06-30', '2019-01-05', '2018-12-15', '2018-06-16', '2018-05-26')) or (app_id = 20600008374430 and date in ('2017-12-23')) or (app_id = 20600009254371 and date in ('2018-12-01', '2018-11-10', '2018-10-27', '2018-04-28', '2018-04-07', '2018-11-03', '2018-10-13', '2018-04-14', '2018-05-05', '2018-10-20', '2018-03-10', '2018-10-06', '2018-03-17', '2018-02-24', '2018-09-01', '2018-03-03', '2018-09-08', '2018-08-18', '2018-08-25', '2018-08-04', '2018-08-11', '2018-07-28', '2019-01-05', '2018-11-24')) or (app_id = 1118448120 and date in ('2017-05-06', '2017-04-15', '2018-12-08', '2018-11-17', '2018-10-27', '2017-10-28', '2017-10-07', '2017-09-16', '2018-03-31', '2018-03-10', '2017-03-11', '2017-02-18', '2017-01-28', '2018-09-22', '2018-09-01', '2017-08-12', '2017-07-22', '2018-02-03', '2018-01-13', '2018-07-28', '2018-07-07', '2017-06-17', '2017-05-27', '2018-12-29', '2017-12-09', '2017-11-18', '2018-06-02', '2018-05-12', '2018-04-21', '2017-04-22', '2017-04-01', '2018-11-03', '2018-10-13', '2017-10-14', '2017-09-23', '2017-09-02', '2018-03-17', '2018-02-24', '2017-02-04', '2017-01-14', '2018-09-08', '2018-08-18', '2017-07-29', '2017-07-08', '2018-01-20', '2017-12-30', '2018-07-14', '2018-06-23', '2017-06-03', '2017-05-13', '2019-01-05', '2018-12-15', '2018-11-24', '2017-11-25', '2017-11-04', '2018-05-19', '2018-04-28', '2018-04-07', '2017-04-08', '2017-03-18', '2017-02-25', '2018-10-20', '2018-09-29', '2017-09-30', '2017-09-09', '2017-08-19', '2018-03-03', '2018-02-10', '2017-01-21', '2018-08-25', '2018-08-04', '2017-07-15', '2017-06-24', '2018-01-06', '2017-12-16', '2018-06-30', '2018-06-09', '2017-05-20', '2017-04-29', '2018-12-22', '2018-12-01', '2018-11-10', '2017-11-11', '2017-10-21', '2018-05-05', '2018-04-14', '2018-03-24', '2017-03-25', '2017-03-04', '2017-02-11', '2018-10-06', '2018-09-15', '2017-08-26', '2017-08-05', '2018-02-17', '2018-01-27', '2017-01-07', '2018-08-11', '2018-07-21', '2017-07-01', '2017-06-10', '2017-12-23', '2017-12-02', '2018-06-16', '2018-05-26')) or (app_id = 20600007316594 and date in ('2017-07-22', '2018-04-21', '2018-08-04', '2018-04-28', '2017-07-15', '2018-05-05', '2018-04-14', '2017-09-30', '2017-08-26')) or (app_id = 1337585357 and date in ('2018-12-22', '2018-12-01', '2018-11-10', '2018-12-08', '2018-11-17', '2018-10-27', '2018-11-03', '2018-10-13', '2018-10-20', '2018-09-29', '2018-10-06', '2018-09-15', '2018-09-22', '2018-09-01', '2018-09-08', '2018-08-18', '2018-08-25', '2018-08-11', '2018-12-29', '2019-01-05', '2018-12-15', '2018-11-24')) or (app_id = 1167445123 and date in ('2017-05-06', '2017-04-15', '2018-11-17', '2018-10-27', '2017-10-28', '2017-10-07', '2017-09-16', '2018-03-31', '2018-03-10', '2017-03-11', '2017-02-18', '2017-01-28', '2018-09-22', '2018-09-01', '2017-08-12', '2017-07-22', '2018-02-03', '2018-01-13', '2018-07-28', '2018-07-07', '2017-06-17', '2017-05-27', '2017-12-09', '2017-11-18', '2018-06-02', '2018-05-12', '2018-04-21', '2017-04-22', '2017-04-01', '2018-11-03', '2018-10-13', '2017-10-14', '2017-09-23', '2017-09-02', '2018-03-17', '2018-02-24', '2017-02-04', '2017-01-14', '2018-09-08', '2018-08-18', '2017-07-29', '2017-07-08', '2018-01-20', '2017-12-30', '2018-07-14', '2018-06-23', '2017-06-03', '2017-05-13', '2018-11-24', '2017-11-25', '2017-11-04', '2018-05-19', '2018-04-28', '2018-04-07', '2017-04-08', '2017-03-18', '2017-02-25', '2018-10-20', '2018-09-29', '2017-09-30', '2017-09-09', '2017-08-19', '2018-03-03', '2018-02-10', '2017-01-21', '2018-08-25', '2018-08-04', '2017-07-15', '2017-06-24', '2018-01-06', '2017-12-16', '2018-06-30', '2018-06-09', '2017-05-20', '2017-04-29', '2018-12-01', '2018-11-10', '2017-11-11', '2017-10-21', '2018-05-05', '2018-04-14', '2018-03-24', '2017-03-25', '2017-03-04', '2017-02-11', '2018-10-06', '2018-09-15', '2017-08-26', '2017-08-05', '2018-02-17', '2018-01-27', '2017-01-07', '2018-08-11', '2018-07-21', '2017-07-01', '2017-06-10', '2017-12-23', '2017-12-02', '2018-06-16', '2018-05-26')) or (app_id = 1436119504 and date in ('2018-12-22', '2018-12-01', '2018-11-10', '2018-12-08', '2018-11-17', '2018-11-03', '2018-12-29', '2019-01-05', '2018-12-15', '2018-11-24')) or (app_id = 912304218 and date in ('2017-05-06', '2017-04-15', '2018-12-08', '2018-11-17', '2018-10-27', '2017-10-28', '2017-10-07', '2017-09-16', '2018-03-31', '2018-03-10', '2017-03-11', '2017-02-18', '2017-01-28', '2018-09-22', '2018-09-01', '2017-08-12', '2017-07-22', '2018-02-03', '2018-01-13', '2018-07-28', '2018-07-07', '2017-06-17', '2017-05-27', '2018-12-29', '2017-12-09', '2017-11-18', '2018-06-02', '2018-05-12', '2018-04-21', '2017-04-22', '2017-04-01', '2018-11-03', '2018-10-13', '2017-10-14', '2017-09-23', '2017-09-02', '2018-03-17', '2018-02-24', '2017-02-04', '2017-01-14', '2018-09-08', '2018-08-18', '2017-07-29', '2017-07-08', '2018-01-20', '2017-12-30', '2018-07-14', '2018-06-23', '2017-06-03', '2017-05-13', '2019-01-05', '2018-12-15', '2018-11-24', '2017-11-25', '2017-11-04', '2018-05-19', '2018-04-28', '2018-04-07', '2017-04-08', '2017-03-18', '2017-02-25', '2018-10-20', '2018-09-29', '2017-09-30', '2017-09-09', '2017-08-19', '2018-03-03', '2018-02-10', '2017-01-21', '2018-08-25', '2018-08-04', '2017-07-15', '2017-06-24', '2018-01-06', '2017-12-16', '2018-06-30', '2018-06-09', '2017-05-20', '2017-04-29', '2018-12-22', '2018-12-01', '2018-11-10', '2017-11-11', '2017-10-21', '2018-05-05', '2018-04-14', '2018-03-24', '2017-03-25', '2017-03-04', '2017-02-11', '2018-10-06', '2018-09-15', '2017-08-26', '2017-08-05', '2018-02-17', '2018-01-27', '2017-01-07', '2018-08-11', '2018-07-21', '2017-07-01', '2017-06-10', '2017-12-23', '2017-12-02', '2018-06-16', '2018-05-26')) or (app_id = 918499617 and date in ('2017-05-06', '2017-04-15', '2018-12-08', '2018-11-17', '2018-10-27', '2017-10-28', '2017-10-07', '2017-09-16', '2018-03-31', '2018-03-10', '2017-03-11', '2017-02-18', '2018-09-22', '2018-09-01', '2017-08-12', '2017-07-22', '2018-02-03', '2018-01-13', '2018-07-28', '2018-07-07', '2017-06-17', '2017-05-27', '2018-12-29', '2017-12-09', '2017-11-18', '2018-06-02', '2018-05-12', '2018-04-21', '2017-04-22', '2017-04-01', '2018-11-03', '2018-10-13', '2017-10-14', '2017-09-23', '2017-09-02', '2018-03-17', '2018-02-24', '2017-02-04', '2017-01-14', '2018-09-08', '2018-08-18', '2017-07-29', '2017-07-08', '2018-01-20', '2017-12-30', '2018-07-14', '2018-06-23', '2017-06-03', '2017-05-13', '2019-01-05', '2018-12-15', '2018-11-24', '2017-11-25', '2017-11-04', '2018-05-19', '2018-04-28', '2018-04-07', '2017-04-08', '2017-03-18', '2017-02-25', '2018-10-20', '2018-09-29', '2017-09-30', '2017-09-09', '2017-08-19', '2018-03-03', '2018-02-10', '2018-08-25', '2018-08-04', '2017-07-15', '2017-06-24', '2018-01-06', '2017-12-16', '2018-06-30', '2018-06-09', '2017-05-20', '2017-04-29', '2018-12-22', '2018-12-01', '2018-11-10', '2017-11-11', '2017-10-21', '2018-05-05', '2018-04-14', '2018-03-24', '2017-03-25', '2017-03-04', '2017-02-11', '2018-10-06', '2018-09-15', '2017-08-26', '2017-08-05', '2018-02-17', '2018-01-27', '2017-01-07', '2018-08-11', '2018-07-21', '2017-07-01', '2017-06-10', '2017-12-23', '2017-12-02', '2018-06-16', '2018-05-26')) or (app_id = 1405449422 and date in ('2018-12-22', '2018-12-01', '2018-11-10', '2018-12-08', '2018-11-17', '2018-10-27', '2018-10-13', '2018-10-20', '2018-09-29', '2018-10-06', '2018-09-15', '2018-09-22', '2018-09-08', '2018-08-18', '2018-08-25', '2018-08-04', '2018-08-11', '2018-07-21', '2018-07-28', '2018-07-14', '2018-12-29', '2018-11-24')) or (app_id = 1147162181 and date in ('2017-05-06', '2017-04-15', '2018-12-08', '2018-11-17', '2018-10-27', '2017-10-28', '2017-10-07', '2017-09-16', '2018-03-31', '2018-03-10', '2017-03-11', '2017-02-18', '2017-01-28', '2018-09-22', '2018-09-01', '2017-08-12', '2017-07-22', '2018-02-03', '2018-01-13', '2018-07-28', '2018-07-07', '2017-06-17', '2017-05-27', '2018-12-29', '2017-12-09', '2017-11-18', '2018-06-02', '2018-05-12', '2018-04-21', '2017-04-22', '2017-04-01', '2018-11-03', '2018-10-13', '2017-10-14', '2017-09-23', '2017-09-02', '2018-03-17', '2018-02-24', '2017-02-04', '2017-01-14', '2018-09-08', '2018-08-18', '2017-07-29', '2017-07-08', '2018-01-20', '2017-12-30', '2018-07-14', '2018-06-23', '2017-06-03', '2017-05-13', '2019-01-05', '2018-12-15', '2018-11-24', '2017-11-25', '2017-11-04', '2018-05-19', '2018-04-28', '2018-04-07', '2017-04-08', '2017-03-18', '2017-02-25', '2018-10-20', '2018-09-29', '2017-09-30', '2017-09-09', '2017-08-19', '2018-03-03', '2018-02-10', '2017-01-21', '2018-08-25', '2018-08-04', '2017-07-15', '2017-06-24', '2018-01-06', '2017-12-16', '2018-06-30', '2018-06-09', '2017-05-20', '2017-04-29', '2018-12-22', '2018-12-01', '2018-11-10', '2017-11-11', '2017-10-21', '2018-05-05', '2018-04-14', '2018-03-24', '2017-03-25', '2017-03-04', '2017-02-11', '2018-10-06', '2018-09-15', '2017-08-26', '2017-08-05', '2018-02-17', '2018-01-27', '2017-01-07', '2018-08-11', '2018-07-21', '2017-07-01', '2017-06-10', '2017-12-23', '2017-12-02', '2018-06-16', '2018-05-26')) or (app_id = 20600005498017 and date in ('2017-05-06', '2018-12-08', '2018-11-17', '2018-10-27', '2017-10-28', '2017-10-07', '2017-09-16', '2018-03-31', '2018-03-10', '2018-09-22', '2018-09-01', '2017-08-12', '2017-07-22', '2018-02-03', '2018-01-13', '2018-07-28', '2018-07-07', '2017-06-17', '2017-05-27', '2018-12-29', '2017-12-09', '2017-11-18', '2018-06-02', '2018-05-12', '2018-04-21', '2018-11-03', '2018-10-13', '2017-10-14', '2017-09-23', '2017-09-02', '2018-03-17', '2018-02-24', '2018-09-08', '2018-08-18', '2017-07-29', '2017-07-08', '2018-01-20', '2017-12-30', '2018-07-14', '2018-06-23', '2017-06-03', '2017-05-13', '2019-01-05', '2018-12-15', '2018-11-24', '2017-11-25', '2017-11-04', '2018-05-19', '2018-04-28', '2018-04-07', '2018-10-20', '2018-09-29', '2017-09-30', '2017-09-09', '2017-08-19', '2018-03-03', '2018-02-10', '2018-08-25', '2018-08-04', '2017-07-15', '2017-06-24', '2018-01-06', '2017-12-16', '2018-06-30', '2018-06-09', '2017-05-20', '2017-04-29', '2018-12-22', '2018-12-01', '2018-11-10', '2017-11-11', '2017-10-21', '2018-05-05', '2018-04-14', '2018-03-24', '2018-10-06', '2018-09-15', '2017-08-26', '2017-08-05', '2018-02-17', '2018-01-27', '2018-08-11', '2018-07-21', '2017-07-01', '2017-06-10', '2017-12-23', '2017-12-02', '2018-06-16', '2018-05-26')) or (app_id = 20600010701988 and date in ('2018-12-22', '2018-12-01', '2018-11-10', '2018-12-08', '2018-11-17', '2018-10-27', '2018-11-03', '2018-10-13', '2018-10-20', '2018-12-29', '2019-01-05', '2018-12-15', '2018-11-24')) or (app_id = 20600005057247 and date in ('2017-11-25', '2017-11-04', '2018-12-08', '2018-11-17', '2018-10-27', '2017-10-28', '2018-10-20', '2018-09-29', '2018-03-10', '2018-09-22', '2018-09-01', '2018-03-03', '2018-02-10', '2018-08-25', '2018-02-03', '2018-07-07', '2018-01-06', '2018-12-29', '2017-12-09', '2017-11-18', '2018-12-22', '2018-12-01', '2018-11-10', '2018-06-02', '2017-11-11', '2018-11-03', '2018-10-13', '2018-03-24', '2018-10-06', '2018-09-15', '2018-03-17', '2018-02-24', '2018-09-08', '2018-08-18', '2018-02-17', '2018-01-27', '2018-07-21', '2017-12-30', '2018-07-14', '2018-06-23', '2017-12-23', '2017-12-02', '2019-01-05', '2018-12-15', '2018-11-24')) or (app_id = 20600005202826 and date in ('2017-07-01', '2017-03-11')) or (app_id = 20600004178334 and date in ('2017-05-06', '2017-04-15', '2018-12-08', '2018-11-17', '2018-10-27', '2017-10-28', '2017-10-07', '2017-09-16', '2018-03-31', '2018-03-10', '2017-03-11', '2017-02-18', '2017-01-28', '2018-09-22', '2018-09-01', '2017-08-12', '2017-07-22', '2018-02-03', '2018-01-13', '2018-07-28', '2018-07-07', '2017-06-17', '2017-05-27', '2018-12-29', '2017-12-09', '2017-11-18', '2018-06-02', '2018-05-12', '2018-04-21', '2017-04-22', '2017-04-01', '2018-11-03', '2018-10-13', '2017-10-14', '2017-09-23', '2017-09-02', '2018-03-17', '2018-02-24', '2017-02-04', '2017-01-14', '2018-09-08', '2018-08-18', '2017-07-29', '2017-07-08', '2018-01-20', '2017-12-30', '2018-07-14', '2018-06-23', '2017-06-03', '2017-05-13', '2019-01-05', '2018-12-15', '2018-11-24', '2017-11-25', '2017-11-04', '2018-05-19', '2018-04-28', '2018-04-07', '2017-04-08', '2017-03-18', '2017-02-25', '2018-10-20', '2018-09-29', '2017-09-30', '2017-09-09', '2017-08-19', '2018-03-03', '2018-02-10', '2017-01-21', '2018-08-25', '2018-08-04', '2017-07-15', '2017-06-24', '2018-01-06', '2017-12-16', '2018-06-30', '2018-06-09', '2017-05-20', '2017-04-29', '2018-12-22', '2018-12-01', '2018-11-10', '2017-11-11', '2017-10-21', '2018-05-05', '2018-04-14', '2018-03-24', '2017-03-25', '2017-03-04', '2017-02-11', '2018-10-06', '2018-09-15', '2017-08-26', '2017-08-05', '2018-02-17', '2018-01-27', '2017-01-07', '2018-08-11', '2018-07-21', '2017-07-01', '2017-06-10', '2017-12-23', '2017-12-02', '2018-06-16', '2018-05-26')) or (app_id = 20600007435836 and date in ('2017-10-28', '2017-10-07', '2017-09-16', '2018-03-31', '2018-03-10', '2018-09-01', '2017-08-12', '2017-07-22', '2018-02-03', '2018-01-13', '2018-07-28', '2018-07-07', '2017-06-17', '2017-05-27', '2017-12-09', '2017-11-18', '2018-06-02', '2018-05-12', '2018-04-21', '2017-10-14', '2017-09-23', '2017-09-02', '2018-03-17', '2018-02-24', '2018-09-08', '2018-08-18', '2017-07-29', '2017-07-08', '2018-01-20', '2017-12-30', '2018-07-14', '2018-06-23', '2017-06-03', '2017-11-25', '2017-11-04', '2018-05-19', '2018-04-28', '2018-04-07', '2017-09-30', '2017-09-09', '2017-08-19', '2018-03-03', '2018-02-10', '2018-08-25', '2018-08-04', '2017-07-15', '2017-06-24', '2018-01-06', '2017-12-16', '2018-06-30', '2018-06-09', '2017-05-20', '2017-11-11', '2017-10-21', '2018-05-05', '2018-04-14', '2018-03-24', '2018-09-15', '2017-08-26', '2017-08-05', '2018-02-17', '2018-01-27', '2018-08-11', '2018-07-21', '2017-07-01', '2017-06-10', '2017-12-23', '2017-12-02', '2018-06-16', '2018-05-26')) or (app_id = 20600010055403 and date in ('2018-08-04', '2018-07-28', '2018-07-07', '2018-09-29', '2018-07-14', '2018-06-30')) or (app_id = 1287987924 and date in ('2018-12-22', '2018-12-01', '2018-11-17', '2018-11-03', '2018-10-20', '2018-10-06', '2018-09-15', '2018-09-22', '2018-09-01', '2018-09-08', '2018-08-25', '2018-08-11', '2018-07-21', '2018-07-28', '2018-07-07', '2018-07-14', '2018-06-23', '2018-12-29', '2018-12-15', '2018-06-16')) or (app_id = 1327208600 and date in ('2018-11-24', '2018-12-08', '2018-11-17', '2018-05-19', '2018-10-27', '2018-04-28', '2018-04-07', '2018-10-20', '2018-09-29', '2018-03-31', '2018-03-10', '2018-09-22', '2018-09-01', '2018-03-03', '2018-02-10', '2018-08-25', '2018-08-04', '2018-02-03', '2018-01-13', '2018-07-28', '2018-07-07', '2018-01-06', '2018-12-29', '2018-06-30', '2018-06-09', '2018-12-22', '2018-12-01', '2018-06-02', '2018-05-12', '2018-04-21', '2018-11-10', '2018-11-03', '2018-05-05', '2018-10-13', '2018-03-24', '2018-04-14', '2018-10-06', '2018-09-15', '2018-03-17', '2018-02-24', '2018-09-08', '2018-08-18', '2018-02-17', '2018-01-27', '2018-08-11', '2018-07-21', '2018-01-20', '2018-07-14', '2018-06-23', '2019-01-05', '2018-12-15', '2018-06-16', '2018-05-26')) or (app_id = 1402727988 and date in ('2018-12-22', '2018-12-08', '2018-12-29', '2019-01-05', '2018-12-15')) or (app_id = 1441506897 and date in ('2018-12-22', '2018-12-29', '2019-01-05')) or (app_id = 20600001544826 and date in ('2017-05-06', '2017-04-15', '2018-12-08', '2018-11-17', '2018-10-27', '2017-10-28', '2017-10-07', '2017-09-16', '2018-03-31', '2018-03-10', '2017-03-11', '2017-02-18', '2017-01-28', '2018-09-22', '2018-09-01', '2017-08-12', '2017-07-22', '2018-02-03', '2018-01-13', '2018-07-28', '2018-07-07', '2017-06-17', '2017-05-27', '2018-12-29', '2017-12-09', '2017-11-18', '2018-06-02', '2018-05-12', '2018-04-21', '2017-04-22', '2017-04-01', '2018-11-03', '2018-10-13', '2017-10-14', '2017-09-23', '2017-09-02', '2018-03-17', '2018-02-24', '2017-02-04', '2017-01-14', '2018-09-08', '2018-08-18', '2017-07-29', '2017-07-08', '2018-01-20', '2017-12-30', '2018-07-14', '2018-06-23', '2017-06-03', '2017-05-13', '2019-01-05', '2018-12-15', '2018-11-24', '2017-11-25', '2017-11-04', '2018-05-19', '2018-04-28', '2018-04-07', '2017-04-08', '2017-03-18', '2017-02-25', '2018-10-20', '2018-09-29', '2017-09-30', '2017-09-09', '2017-08-19', '2018-03-03', '2018-02-10', '2017-01-21', '2018-08-25', '2018-08-04', '2017-07-15', '2017-06-24', '2018-01-06', '2017-12-16', '2018-06-30', '2018-06-09', '2017-05-20', '2017-04-29', '2018-12-22', '2018-12-01', '2018-11-10', '2017-11-11', '2017-10-21', '2018-05-05', '2018-04-14', '2018-03-24', '2017-03-25', '2017-03-04', '2017-02-11', '2018-10-06', '2018-09-15', '2017-08-26', '2017-08-05', '2018-02-17', '2018-01-27', '2017-01-07', '2018-08-11', '2018-07-21', '2017-07-01', '2017-06-10', '2017-12-23', '2017-12-02', '2018-06-16', '2018-05-26')) or (app_id = 992805875 and date in ('2018-12-08', '2018-11-17', '2018-10-27', '2018-10-20', '2018-09-29', '2018-09-22', '2018-09-01', '2018-08-25', '2018-08-04', '2018-07-28', '2018-07-07', '2018-12-29', '2018-06-30', '2018-12-22', '2018-12-01', '2018-11-10', '2018-11-03', '2018-10-13', '2018-10-06', '2018-09-15', '2018-09-08', '2018-08-18', '2018-08-11', '2018-07-21', '2018-07-14', '2019-01-05', '2018-12-15', '2018-11-24')) or (app_id = 20600005403566 and date in ('2017-05-06', '2017-04-15', '2018-11-17', '2018-10-27', '2017-10-28', '2017-10-07', '2017-09-16', '2018-03-31', '2018-03-10', '2017-03-11', '2017-02-18', '2017-01-28', '2018-09-22', '2018-09-01', '2017-08-12', '2017-07-22', '2018-02-03', '2018-01-13', '2018-07-28', '2018-07-07', '2017-06-17', '2017-05-27', '2018-12-29', '2017-12-09', '2017-11-18', '2018-06-02', '2018-05-12', '2018-04-21', '2017-04-22', '2017-04-01', '2018-11-03', '2018-10-13', '2017-10-14', '2017-09-23', '2017-09-02', '2018-03-17', '2018-02-24', '2017-02-04', '2017-01-14', '2018-09-08', '2018-08-18', '2017-07-29', '2017-07-08', '2018-01-20', '2017-12-30', '2018-07-14', '2018-06-23', '2017-06-03', '2017-05-13', '2019-01-05', '2018-12-15', '2018-11-24', '2017-11-25', '2017-11-04', '2018-05-19', '2018-04-28', '2018-04-07', '2017-04-08', '2017-03-18', '2017-02-25', '2018-10-20', '2018-09-29', '2017-09-30', '2017-09-09', '2017-08-19', '2018-03-03', '2018-02-10', '2017-01-21', '2018-08-25', '2018-08-04', '2017-07-15', '2017-06-24', '2018-01-06', '2017-12-16', '2018-06-30', '2018-06-09', '2017-05-20', '2017-04-29', '2018-12-22', '2018-12-01', '2018-11-10', '2017-11-11', '2017-10-21', '2018-05-05', '2018-04-14', '2018-03-24', '2017-03-25', '2017-03-04', '2017-02-11', '2018-10-06', '2018-09-15', '2017-08-26', '2017-08-05', '2018-02-17', '2018-01-27', '2017-01-07', '2018-08-11', '2018-07-21', '2017-07-01', '2017-06-10', '2017-12-23', '2017-12-02', '2018-06-16', '2018-05-26')) or (app_id = 20600010732594 and date in ('2018-12-22', '2018-12-01', '2018-11-10', '2018-12-08', '2018-11-17', '2018-10-27', '2018-11-03', '2018-10-20', '2018-12-29', '2019-01-05', '2018-12-15', '2018-11-24')) or (app_id = 1135820611 and date in ('2017-05-06', '2017-04-15', '2018-12-08', '2018-11-17', '2018-10-27', '2017-10-28', '2017-10-07', '2017-09-16', '2018-03-31', '2018-03-10', '2017-03-11', '2017-02-18', '2017-01-28', '2018-09-22', '2018-09-01', '2017-08-12', '2017-07-22', '2018-02-03', '2018-01-13', '2018-07-28', '2018-07-07', '2017-06-17', '2017-05-27', '2018-12-29', '2017-12-09', '2017-11-18', '2018-06-02', '2018-05-12', '2018-04-21', '2017-04-22', '2017-04-01', '2018-11-03', '2018-10-13', '2017-10-14', '2017-09-23', '2017-09-02', '2018-03-17', '2018-02-24', '2017-02-04', '2017-01-14', '2018-09-08', '2018-08-18', '2017-07-29', '2017-07-08', '2018-01-20', '2017-12-30', '2018-07-14', '2018-06-23', '2017-06-03', '2017-05-13', '2019-01-05', '2018-12-15', '2018-11-24', '2017-11-25', '2017-11-04', '2018-05-19', '2018-04-28', '2018-04-07', '2017-04-08', '2017-03-18', '2017-02-25', '2018-10-20', '2018-09-29', '2017-09-30', '2017-09-09', '2017-08-19', '2018-03-03', '2018-02-10', '2017-01-21', '2018-08-25', '2018-08-04', '2017-07-15', '2017-06-24', '2018-01-06', '2017-12-16', '2018-06-30', '2018-06-09', '2017-05-20', '2017-04-29', '2018-12-22', '2018-12-01', '2018-11-10', '2017-11-11', '2017-10-21', '2018-05-05', '2018-04-14', '2018-03-24', '2017-03-25', '2017-03-04', '2017-02-11', '2018-10-06', '2018-09-15', '2017-08-26', '2017-08-05', '2018-02-17', '2018-01-27', '2017-01-07', '2018-08-11', '2018-07-21', '2017-07-01', '2017-06-10', '2017-12-23', '2017-12-02', '2018-06-16', '2018-05-26')) or (app_id = 1364480289 and date in ('2018-12-22', '2018-12-01', '2018-11-10', '2018-12-08', '2018-11-17', '2018-10-27', '2018-11-03', '2018-10-13', '2018-10-20', '2018-09-29', '2018-10-06', '2018-09-15', '2018-09-22', '2018-09-01', '2018-09-08', '2018-08-18', '2018-08-25', '2018-08-11', '2018-12-29', '2019-01-05', '2018-12-15', '2018-11-24')) or (app_id = 507243750 and date in ('2017-11-04', '2017-01-21', '2017-10-28', '2017-12-16', '2018-06-23', '2017-09-09', '2017-01-28')) or (app_id = 20600008976933 and date in ('2018-12-08', '2018-05-19', '2018-04-28', '2018-04-07', '2018-09-29', '2018-03-10', '2018-09-22', '2018-03-03', '2018-08-25', '2018-08-04', '2018-07-28', '2018-07-07', '2018-12-29', '2018-06-30', '2018-12-22', '2018-06-02', '2018-05-12', '2018-05-05', '2018-04-14', '2018-03-24', '2018-09-15', '2018-03-17', '2018-02-24', '2018-08-18', '2018-02-17', '2018-01-27', '2018-08-11', '2018-01-20', '2018-07-14', '2018-06-23', '2018-12-15', '2018-05-26')) or (app_id = 662012375 and date in ('2017-05-06', '2017-04-15', '2018-12-08', '2018-11-17', '2018-10-27', '2017-10-28', '2017-10-07', '2017-09-16', '2018-03-31', '2018-03-10', '2017-03-11', '2017-02-18', '2017-01-28', '2018-09-22', '2018-09-01', '2017-08-12', '2017-07-22', '2018-02-03', '2018-01-13', '2018-07-28', '2018-07-07', '2017-06-17', '2017-05-27', '2018-12-29', '2017-12-09', '2017-11-18', '2018-06-02', '2018-05-12', '2018-04-21', '2017-04-22', '2017-04-01', '2018-11-03', '2018-10-13', '2017-10-14', '2017-09-23', '2017-09-02', '2018-03-17', '2018-02-24', '2017-02-04', '2017-01-14', '2018-09-08', '2018-08-18', '2017-07-29', '2017-07-08', '2018-01-20', '2017-12-30', '2018-07-14', '2018-06-23', '2017-06-03', '2017-05-13', '2019-01-05', '2018-12-15', '2018-11-24', '2017-11-25', '2017-11-04', '2018-05-19', '2018-04-28', '2018-04-07', '2017-04-08', '2017-03-18', '2017-02-25', '2018-10-20', '2018-09-29', '2017-09-30', '2017-09-09', '2017-08-19', '2018-03-03', '2018-02-10', '2017-01-21', '2018-08-25', '2018-08-04', '2017-07-15', '2017-06-24', '2018-01-06', '2017-12-16', '2018-06-30', '2018-06-09', '2017-05-20', '2017-04-29', '2018-12-22', '2018-12-01', '2018-11-10', '2017-11-11', '2017-10-21', '2018-05-05', '2018-04-14', '2018-03-24', '2017-03-25', '2017-03-04', '2017-02-11', '2018-10-06', '2018-09-15', '2017-08-26', '2017-08-05', '2018-02-17', '2018-01-27', '2017-01-07', '2018-08-11', '2018-07-21', '2017-07-01', '2017-06-10', '2017-12-23', '2017-12-02', '2018-06-16', '2018-05-26')) or (app_id = 20600004587736 and date in ('2017-05-06', '2017-04-15', '2017-04-22', '2017-04-01', '2017-04-08', '2017-03-18', '2017-02-25', '2017-03-25', '2017-03-04', '2017-02-11', '2017-03-11', '2017-02-18', '2017-01-28', '2017-02-04', '2017-01-14', '2017-01-21', '2017-01-07', '2017-05-27', '2017-06-03', '2017-05-13', '2017-05-20', '2017-04-29')) or (app_id = 20600005869456 and date in ('2017-11-25', '2017-11-04', '2017-05-06', '2017-04-15', '2017-10-28', '2017-10-07', '2017-09-16', '2017-03-18', '2017-02-25', '2017-04-08', '2017-09-30', '2017-09-09', '2017-03-11', '2017-02-18', '2017-01-28', '2017-08-19', '2017-08-12', '2017-07-22', '2017-01-21', '2017-07-15', '2017-06-24', '2017-12-16', '2017-06-17', '2017-05-27', '2017-12-09', '2017-11-18', '2017-05-20', '2017-04-29', '2017-11-11', '2017-10-21', '2017-04-22', '2017-04-01', '2017-10-14', '2017-09-23', '2017-03-25', '2017-03-04', '2017-02-11', '2017-09-02', '2017-08-26', '2017-08-05', '2017-02-04', '2017-01-14', '2017-07-29', '2017-07-08', '2017-01-07', '2017-07-01', '2017-06-10', '2017-12-02', '2017-06-03', '2017-05-13')) or (app_id = 1387810949 and date in ('2018-12-22', '2018-12-01', '2018-11-10', '2018-12-08', '2018-11-17', '2018-10-27', '2018-11-03', '2018-10-13', '2018-10-20', '2018-09-29', '2018-10-06', '2018-09-15', '2018-09-22', '2018-09-01', '2018-09-08', '2018-08-18', '2018-08-25', '2018-08-04', '2018-08-11', '2018-07-21', '2018-07-28', '2018-12-29', '2019-01-05', '2018-12-15', '2018-11-24')) or (app_id = 20600009920721 and date in ('2018-12-08', '2018-11-17', '2018-10-27', '2018-10-20', '2018-09-29', '2018-09-22', '2018-09-01', '2018-08-25', '2018-08-04', '2018-07-28', '2018-07-07', '2018-12-29', '2018-06-30', '2018-06-09', '2018-12-22', '2018-06-02', '2018-11-03', '2018-10-13', '2018-10-06', '2018-09-15', '2018-09-08', '2018-08-18', '2018-08-11', '2018-07-21', '2018-07-14', '2018-06-23', '2019-01-05', '2018-12-15', '2018-11-24', '2018-06-16')) or (app_id = 20600001386312 and date in ('2017-05-06', '2017-04-15', '2018-12-08', '2018-11-17', '2018-10-27', '2017-10-28', '2017-10-07', '2017-09-16', '2018-03-31', '2018-03-10', '2017-03-11', '2017-02-18', '2017-01-28', '2018-09-22', '2018-09-01', '2017-08-12', '2017-07-22', '2018-02-03', '2018-01-13', '2018-07-28', '2018-07-07', '2017-06-17', '2017-05-27', '2018-12-29', '2017-12-09', '2017-11-18', '2018-06-02', '2018-05-12', '2018-04-21', '2017-04-22', '2017-04-01', '2018-11-03', '2018-10-13', '2017-10-14', '2017-09-23', '2017-09-02', '2018-03-17', '2018-02-24', '2017-02-04', '2017-01-14', '2018-09-08', '2018-08-18', '2017-07-29', '2017-07-08', '2018-01-20', '2017-12-30', '2018-07-14', '2018-06-23', '2017-06-03', '2017-05-13', '2019-01-05', '2018-12-15', '2018-11-24', '2017-11-25', '2017-11-04', '2018-05-19', '2018-04-28', '2018-04-07', '2017-04-08', '2017-03-18', '2017-02-25', '2018-10-20', '2018-09-29', '2017-09-30', '2017-09-09', '2017-08-19', '2018-03-03', '2018-02-10', '2017-01-21', '2018-08-25', '2018-08-04', '2017-07-15', '2017-06-24', '2018-01-06', '2017-12-16', '2018-06-30', '2018-06-09', '2017-05-20', '2017-04-29', '2018-12-22', '2018-12-01', '2018-11-10', '2017-11-11', '2017-10-21', '2018-05-05', '2018-04-14', '2018-03-24', '2017-03-25', '2017-03-04', '2017-02-11', '2018-10-06', '2018-09-15', '2017-08-26', '2017-08-05', '2018-02-17', '2018-01-27', '2017-01-07', '2018-08-11', '2018-07-21', '2017-07-01', '2017-06-10', '2017-12-23', '2017-12-02', '2018-06-16', '2018-05-26')) or (app_id = 20600009346979 and date in ('2018-05-26', '2018-12-08', '2018-11-17', '2018-10-27', '2018-04-28', '2018-10-20', '2018-09-01', '2018-08-25', '2018-08-04', '2018-07-28', '2018-07-07', '2018-12-29', '2018-06-30', '2018-12-22', '2018-12-01', '2018-06-02', '2018-11-10', '2018-05-12', '2018-11-03', '2018-09-08', '2018-08-18', '2018-08-11', '2018-07-21', '2018-07-14', '2018-06-23', '2019-01-05', '2018-12-15', '2018-11-24', '2018-06-16')) or (app_id = 1440025446 and date in ('2018-12-22', '2018-12-29', '2019-01-05')) or (app_id = 20600003064195 and date in ('2017-05-06', '2017-04-15', '2018-12-08', '2018-11-17', '2018-10-27', '2017-10-28', '2017-10-07', '2017-09-16', '2018-03-31', '2018-03-10', '2017-03-11', '2017-02-18', '2017-01-28', '2018-09-22', '2018-09-01', '2017-08-12', '2017-07-22', '2018-02-03', '2018-01-13', '2018-07-28', '2018-07-07', '2017-06-17', '2017-05-27', '2018-12-29', '2017-12-09', '2017-11-18', '2018-06-02', '2018-05-12', '2018-04-21', '2017-04-22', '2017-04-01', '2018-11-03', '2018-10-13', '2017-10-14', '2017-09-23', '2017-09-02', '2018-03-17', '2018-02-24', '2017-02-04', '2017-01-14', '2018-09-08', '2018-08-18', '2017-07-29', '2017-07-08', '2018-01-20', '2017-12-30', '2018-07-14', '2018-06-23', '2017-06-03', '2017-05-13', '2019-01-05', '2018-12-15', '2018-11-24', '2017-11-25', '2017-11-04', '2018-05-19', '2018-04-28', '2018-04-07', '2017-04-08', '2017-03-18', '2017-02-25', '2018-10-20', '2018-09-29', '2017-09-30', '2017-09-09', '2017-08-19', '2018-03-03', '2018-02-10', '2017-01-21', '2018-08-25', '2018-08-04', '2017-07-15', '2017-06-24', '2018-01-06', '2017-12-16', '2018-06-30', '2018-06-09', '2017-05-20', '2017-04-29', '2018-12-22', '2018-12-01', '2018-11-10', '2017-11-11', '2017-10-21', '2018-05-05', '2018-04-14', '2018-03-24', '2017-03-25', '2017-03-04', '2017-02-11', '2018-10-06', '2018-09-15', '2017-08-26', '2017-08-05', '2018-02-17', '2018-01-27', '2017-01-07', '2018-08-11', '2018-07-21', '2017-07-01', '2017-06-10', '2017-12-23', '2017-12-02', '2018-06-16', '2018-05-26')))


limit 10
;
rollback;
EOF

In [0]:

insert_weekly_df.show(500, False)

In [0]:

diff_df.filter("app_id = 20600006880641").show(10, False)

In [0]:
%%sh
PGPASSWORD='PZQYyjRHCXZ23LVQ' psql -h 10.2.6.141  -U citus_bdp_prod_int_datapipeline -d aa_store_db -p 5432 << EOF
BEGIN;
select *
from store.store_est_t_w_fact_v2
where 
    granularity = 'weekly'
    and app_id = 20600006880641
    AND date between '2015-01-07' and '2016-08-07'
limit 10
;
rollback;
EOF

In [0]:

import datetime
from bdce.common.utils import update_application_code
update_application_code(spark, 'BDP-PROD-INT-DATAPIPELINE', 'usage_basic_kpi_migration')
import time
import traceback
from aadatapipelinecore.core.error import AAIntDataPipelineCoreError
from aadatapipelinecore.core.loader.citus import _is_base_table_exist, _db_config, _create_base_table, _create_index, \
    _create_partition_table, _insert, logger, _validate_storage_definition
from aadatapipelinecore.core.urn import Urn
from aadatapipelinecore.core.utils import semaphore
from aadatapipelinecore.core.utils.date_ import daterange, str_date
from aadatapipelinecore.core.utils.spark import date_to_string
from pyspark import StorageLevel
from pyspark.sql import Row
from aadatapipelinecore.core.utils.identifier import atomic_id
from aadatapipelinecore.core.fs.driver import raw, unified
from aadatapipelinecore.core.pipeline import schema, type_
from aadatapipelinecore.core import loader

def write(urn, df, storage_definition, delete_sql):
    cnt = df.count()
    if not cnt:
        return
    df.persist(StorageLevel.MEMORY_AND_DISK)
    try:
        # semaphore_.acquire()
        start = time.time()
        dto_list = [
        unified.UnifiedDTO(
            dataframe=df,
            model=schema.DataModel(
                type_="fact",
                partition_by=['granularity', "date", "device_code"],
                primary_key=["granularity", "device_code", "country_code", "date", "app_id", "publisher_id", "company_id", "parent_company_id"],
                dimensions=["granularity", "device_code", "country_code", "date", "app_id", "publisher_id", "company_id", "parent_company_id"],
                data_granularity=[type_.DataGranularity.WEEKLY]
            )
        )]
        loader.write(spark, urn, storage_definition, dto_list, chunk_size=1000000, delete_statement=delete_sql)
        logger.info("db write take: " + str(time.time() - start))
    except Exception():
        task_trace = traceback.format_exc()
        logger.error(task_trace)
    finally:
        pass
    df.unpersist()

urn = Urn(
        namespace='aa.store.app-est-dna-log-pre-aggr.v1',
        identifier=atomic_id()
    )

write(urn, insert_weekly_df.repartition(40), storage_definition, delete_sql)

In [0]:
%%sh

PGPASSWORD='PZQYyjRHCXZ23LVQ' psql -h 10.2.6.141 -p 5432 -Ucitus_bdp_prod_int_datapipeline -d aa_store_db << EOF
SET search_path=store;

begin;

create  table store_est_fact_temp_v1_temp as select * from store_est_fact_temp_v1  limit 1;

SELECT
 create_distributed_table(
    'store.store_est_fact_temp_v1_temp',
    'app_id',
    colocate_with => 'store.store_est_fact_temp_v1'
);

UPDATE store_est_fact_temp_v1 AS v 

SET 
    publisher_id = s.publisher_id,
    company_id = s.company_id,
    parent_company_id = s.parent_company_id

FROM store_est_fact_temp_v1_temp AS s
WHERE 
 v.app_id = s.app_id
and v.date = s.date
and v.country_code = s.country_code
and v.device_code = s.device_code;

--and v.date in (select distinct date from store_est_fact_temp_v1_temp)
--and v.device_code in (select distinct device_code from store_est_fact_temp_v1_temp)
--and v.country_code in (select distinct country_code from store_est_fact_temp_v1_temp)
--and v.app_id in (select distinct app_id from store_est_fact_temp_v1_temp)

rollback;

In [0]:

daily_df.unpersist()
daily_df = insert_daily_df.where("date between '2017-01-01' and '2017-01-31'").repartition(40).cache()
print daily_df.count()


In [0]:


from bdce.common.utils import update_application_code
update_application_code(spark, 'BDP-PROD-INT-DATAPIPELINE', 'store_script')


from  aadatapipelinecore.core.loader.citus import _clean_temp_tables
from aadatapipelinecore.core.loader.citus import _create_temp_table
from aadatapipelinecore.core.loader.base.pg import execute_sql
from aadatapipelinecore.core.loader.citus import _db_config
from aadatapipelinecore.core.urn import Urn
from aadatapipelinecore.core.utils.identifier import atomic_id

import time
import traceback
from aadatapipelinecore.core.error import AAIntDataPipelineCoreError
from aadatapipelinecore.core.loader.citus import _is_base_table_exist, _db_config, _create_base_table, _create_index, \
    _create_partition_table, logger, _validate_storage_definition

from aadatapipelinecore.core.utils import semaphore
from aadatapipelinecore.core.utils.date_ import daterange, str_date
from aadatapipelinecore.core.utils.spark import date_to_string
from pyspark import StorageLevel
from pyspark.sql import Row

from aadatapipelinecore.core.fs.driver import raw, unified
from aadatapipelinecore.core.pipeline import schema, type_
from aadatapipelinecore.core import loader
from copy import deepcopy


UPDATE_COLUMNS = """


UPDATE %(schema)s.%(table)s AS v 

SET 
    publisher_id = s.publisher_id,
    company_id = s.company_id,
    parent_company_id = s.parent_company_id

FROM %(schema)s.%(temp_table)s AS s
WHERE 
    v.app_id = s.app_id
and v.date = s.date
and v.granularity = s.granularity
and v.country_code = s.country_code
and v.device_code = s.device_code
;
"""



import csv
import traceback
from StringIO import StringIO

import psycopg2

from aadatapipelinecore.core.constants import DISABLE_PRIMARY_KEY_FOR_QUERY_COLUMN
from aadatapipelinecore.core.error import AAIntDataPipelineCoreError
from aadatapipelinecore.core.loader.base.pg import _dsn, _to_pg_type
from aadatapipelinecore.core.loader.citus import _db_config
from aadatapipelinecore.core.utils import semaphore, list_
from aadatapipelinecore.core.utils.semaphore import _normalize_identifier


def _insert(partition_id, urn, rows, storage_definition, chunk_size):
    """
    Parallel copy data into temp table
    :param partition_id: unique partition identifier
    :type partition_id: str
    :param urn: please check aa-int-data-pipeline/aaintdatapipeline/core/urn.py for more details
    :type urn: aadatapipelinecore.core.urn.Urn
    :param rows: rows that need to be copied
    :type rows: list
    :param storage_definition: storage_definition
    :type storage_definition: dict
    :param chunk_size: count of rows that will be copied in one sql
    :type chunk_size: int
    :return: list of chunk count
    :rtype: list
    """
    config = _db_config(urn, storage_definition)
    dsn_ = _dsn(config)
    conn = None
    cursor = None
    result = 0
    io = None
    trace_str = None

    try:
        config = _db_config(urn, storage_definition)

        # semaphore_name = config['semaphore_name']
        # semaphore_capacity = config['connection_capacity']
        # semaphore_timeout = config['semaphore_timeout']
        identifier = _normalize_identifier(urn.identifier, partition_id)
        # semaphore_ = semaphore.of(
        #     semaphore_name,
        #     urn.namespace,
        #     kwargs={
        #         "value": semaphore_capacity,
        #         "timeout": semaphore_timeout,
        #         "identifier": identifier,
        #         "expire_time": urn.expire_time or 1800,
        #     }
        # )
        try:
            result = pg_insert(
                urn, identifier, storage_definition, None, dsn_, rows, result, chunk_size=chunk_size, temp=True
            )
        except Exception():
            task_trace = traceback.format_exc()
            raise AAIntDataPipelineCoreError(task_trace)
        finally:
            # semaphore_.release()
            pass
    except:
        if conn is not None:
            conn.rollback()
        trace_str = traceback.format_exc()

    finally:
        if cursor is not None:
            cursor.close()
        if conn is not None:
            conn.close()
        if io is not None:
            io.close()
        if trace_str:
            raise AAIntDataPipelineCoreError(trace_str)
        return [result]


def pg_insert(
        urn, identifier, interface, semaphore_, dsn, rows, result, delete_by_sql=None, chunk_size=100000, temp=False
):
    # semaphore_.acquire()
    conn = psycopg2.connect(dsn)
    conn.autocommit = False
    cursor = conn.cursor()

    if delete_by_sql:
        cursor.execute(delete_by_sql)

    rows_chunks = list_.chunks(rows, chunk_size)
    fields = [x['name'] for x in interface['fields']]
    if DISABLE_PRIMARY_KEY_FOR_QUERY_COLUMN in fields:
        fields.remove(DISABLE_PRIMARY_KEY_FOR_QUERY_COLUMN)
    formats = {
        x['name']: (x.get('type'), x.get('format'))
        for x in interface['fields']
    }
    for chunk in rows_chunks:
        io = StringIO()
        writer = csv.writer(io, delimiter='\t')
        for row in chunk:
            values = tuple(
                _to_pg_type(f, row[f], formats)
                for f in fields
            )
            writer.writerow(values)
            result += 1
        io.seek(0)
        table = '.'.join([interface['schema'], interface['temp_table']]) if temp else interface['table']
        cursor.copy_expert(
            r"COPY {} ({}) FROM STDIN "
            r"WITH (FORMAT CSV, DELIMITER E'\t', NULL '\N');".format(table, ",".join(fields)),
            io
        )

    conn.commit()
    return result


def _prepare_tables( urn, storage_definition):
    # print storage_definition
    # print storage_definition['fields']
    config = _db_config(urn, storage_definition)
    storage_definition['schema'] = urn.namespace.split('.')[1]
    from aadatapipelinecore.core.loader.citus import _create_base_table
    from aadatapipelinecore.core.loader.citus import _is_base_table_exist
    from aadatapipelinecore.core.loader.citus import _warn_invalid_param
    # from aadatapipelinecore.core.loader.citus import _create_index
    if _is_base_table_exist(config, storage_definition):
        # print "here"
        _warn_invalid_param(config, storage_definition)
    else:
        print storage_definition['fields']
        _create_base_table(config, storage_definition)
    _create_index(config, storage_definition)
    # storage_definition['temp_table'] = _create_temp_table(config, urn, storage_definition)
    return config
    


def execute_sql(config, sql, params=None):
    from aadatapipelinecore.core.loader.base.pg import connect
    """
    Execute multiple sql in a single transaction
    :param config: db connection config
    :type config: dict
    :param sql:  one sql that need be executed
    :type sql: str
    :param params: parameters of sql query
    :type params: dict
    :return: None
    :rtype: None
    :raises: AAIntDataPipelineCoreError: Connection Error
    """
    conn = None
    cur = None
    trace_str = None
    try:
        start = time.time()
        conn = connect(config)
        conn.autocommit = False
        cur = conn.cursor()
        start = time.time()
        cur.execute("""set citus.propagate_set_commands = 'local';""")
        cur.execute("""set LOCAL max_parallel_workers = 16;""")
        cur.execute("""set LOCAL max_parallel_workers_per_gather = 3;""")
        cur.execute("""set LOCAL enable_nestloop=on;""")
        print sql
        cur.execute(sql)
        conn.commit()
        print time.time() - start
    except:
        if conn is not None:
            conn.rollback()
        trace_str = traceback.format_exc()
    finally:
        if cur is not None:
            cur.close()
        if conn is not None:
            conn.close()
        if trace_str:
            raise AAIntDataPipelineCoreError(trace_str)

def _write(urn, storage_definition, unified_dto_list, chunk_size=100000, delete_statement=None, stats_dict=None):
    from aadatapipelinecore.core.loader.citus import _all_partition_table
    # from aadatapipelinecore.core.loader.base.pg import execute_sql
    _validate_storage_definition(urn.namespace, storage_definition)
    # print  storage_definition['fields']
    storage_definition['schema'] = urn.namespace.split('.')[1]
    # print storage_definition
    # return 
    config = _db_config(urn, storage_definition)
    result_dto_list = []
    for unified_dto in unified_dto_list:
        try:
            df = unified_dto.dataframe
            # print   storage_definition['unique_identifier']['columns']
            _prepare_tables( urn, storage_definition)
            current_partition_tables = _all_partition_table(config, storage_definition)
            if storage_definition.get('partition_by') is not None and not df.rdd.isEmpty():
                _create_partition_table(config, storage_definition, df)
            storage_definition['temp_table'] = storage_definition['table']
            print df.rdd.getNumPartitions()
            start = time.time()
            df.rdd.mapPartitionsWithIndex(
                lambda partition_id, rows:
                _insert(partition_id, urn, rows, storage_definition, chunk_size)
            ).collect()
            temp_time = time.time() 
            print  "temp table",str( temp_time  - start)
            # print storage_definition
            update_storage_definition = deepcopy(storage_definition)
            update_storage_definition['temp_table'] = update_storage_definition['table']
            update_storage_definition['table'] = update_storage_definition['co-located_table']
            sql =  UPDATE_COLUMNS % update_storage_definition
            # print sql
            execute_sql(config,sql)
            print "update", str(time.time() - temp_time)

        finally:
            pass



storage_definition_temp = {
    "engine": "RDB",
    "database": "aa_store_db",
    "cluster": "prod-citus-cluster-4",
    "table": "store_est_fact_tempupdate_v2",
    "co-located_table":"store_est_fact_temp_v1",
    "fields": [
        {"name": "app_id", "type": "bigint", "constraints": "NOT NULL"},
        {"name": "country_code", "type": "character(2)", "constraints": "NOT NULL"},
        {"name": "device_code", "type": "character varying(50)", "constraints": "NOT NULL"},
        {"name": "date", "type": "date", "format": "%Y-%m-%d", "constraints": "NOT NULL"},
        {"name": "granularity", "type": "character varying(8)", "constraints": "NOT NULL"},
        {"name": "publisher_id", "type": "bigint", "constraints": "NOT NULL"},
        {"name": "company_id", "type": "bigint", "constraints": "NOT NULL"},
        {"name": "parent_company_id", "type": "bigint", "constraints": "NOT NULL"}
    ],
    "sharding_count": "64",
    "sharding_key": "app_id",
    "index": [
        {
            "name": "_primary_key",
            "unique": True,
            "method": "btree",
            "column": ["granularity", "device_code", "country_code", "date", "app_id", "publisher_id", "company_id", "parent_company_id"]
        },
        {
            "name": "join_key",
            "method": "btree",
            "column": ["granularity", "device_code", "country_code", "date", "app_id", "publisher_id", "company_id", "parent_company_id"]
        }

    ],
    "partition_by": {
        "name": "date",
        "type": "range",
        "granularity": "monthly"
    },
    "action_on_conflict": "UPDATE"
};



urn = Urn(
    namespace='aa.store.est.v3',
    identifier=atomic_id()
)

unified_dto_list = [
        unified.UnifiedDTO(
            dataframe=daily_df,
            model=schema.DataModel(
                type_="fact",
                partition_by=['granularity', "date", "device_code"],
                primary_key=["granularity", "device_code", "country_code", "date", "app_id", "publisher_id", "company_id", "parent_company_id"],
                dimensions=["granularity", "device_code", "country_code", "date", "app_id", "publisher_id", "company_id", "parent_company_id"],
                data_granularity=[type_.DataGranularity.DAILY]
            )
        )]


# config = _prepare_tables(urn, storage_definition_temp)
start = time.time()
_write(urn, storage_definition_temp, unified_dto_list, chunk_size=100000, delete_statement=None, stats_dict=None)

print "total", str(time.time() - start)


In [0]:

# Copyright (c) 2020 App Annie Inc. All rights reserved.
import datetime
import time
import traceback

from aadatapipelinecore.core.error import AAIntDataPipelineCoreError
from aadatapipelinecore.core.loader.citus import _is_base_table_exist, _db_config, _create_base_table, _create_index, \
    _create_partition_table, _insert, logger, _validate_storage_definition
from aadatapipelinecore.core.utils.date_ import daterange, str_date
from aadatapipelinecore.core import loader
from aadatapipelinecore.core.pipeline import schema
from aadatapipelinecore.core.fs.driver.unified import UnifiedDTO
from aadatapipelinecore.core.urn import Urn
from pyspark import StorageLevel
from pyspark.sql import Row
from aadatapipelinecore.core.utils.identifier import atomic_id
from copy import deepcopy


storage_definition = {
    "engine": "RDB",
    "database": "aa_store_db",
    "cluster": "prod-citus-cluster-4",
    "table": "store_est_fact_temp_v1",
    "fields": [
        {"name": "app_id", "type": "bigint", "constraints": "NOT NULL"},
        {"name": "country_code", "type": "character(2)", "constraints": "NOT NULL"},
        {"name": "device_code", "type": "character varying(50)", "constraints": "NOT NULL"},
        {"name": "date", "type": "date", "format": "%Y-%m-%d", "constraints": "NOT NULL"},
        {"name": "granularity", "type": "character varying(8)", "constraints": "NOT NULL"},
        {"name": "publisher_id", "type": "bigint", "constraints": "NOT NULL"},
        {"name": "company_id", "type": "bigint", "constraints": "NOT NULL"},
        {"name": "parent_company_id", "type": "bigint", "constraints": "NOT NULL"},
        {"name": "est_free_app_download", "type": "int"},
        {"name": "est_paid_app_download", "type": "int"},
        {"name": "est_revenue", "type": "int"},
        {"name": "est_organic_download", "type": "int"},
        {"name": "est_paid_download", "type": "int"}
    ],
    "sharding_count": "64",
    "sharding_key": "app_id",
    "index": [
        {
            "name": "_primary_key",
            "unique": True,
            "method": "btree",
            "column": ["granularity", "device_code", "country_code", "date", "app_id", "publisher_id", "company_id", "parent_company_id"]
        },
        {
            "name": "single_app_device_first",
            "method": "btree",
            "column": ["granularity", "app_id", "date", "device_code", "country_code",
                       "est_free_app_download", "est_paid_app_download", "est_revenue", "est_organic_download",
                       "est_paid_download"]
        },
        {
            "name": "single_app_country_first",
            "method": "btree",
            "column": ["granularity", "app_id", "date", "country_code", "device_code",
                       "est_free_app_download", "est_paid_app_download", "est_revenue", "est_organic_download",
                       "est_paid_download"]
        },
        {
            "name": "join_country_first",
            "method": "btree",
            "column": ["granularity", "date", "country_code", "device_code", "app_id",
                       "est_free_app_download", "est_paid_app_download", "est_revenue", "est_organic_download",
                       "est_paid_download"],
            "include_without_query": "publisher_id, company_id, parent_company_id"
        },
        {
            "name": "join_device_first",
            "method": "btree",
            "column": ["granularity", "date", "device_code", "country_code", "app_id",
                       "est_free_app_download", "est_paid_app_download", "est_revenue", "est_organic_download",
                       "est_paid_download"],
            "include_without_query": "publisher_id, company_id, parent_company_id"
        }

    ],
    "partition_by": {
        "name": "date",
        "type": "range",
        "granularity": "monthly"
    },
    "pre_aggregated_tables": {
        "timeseries": [
            {"agg": "sum", "granularity": "weekly", "table": "store_est_t_w_fact"},
            {"agg": "sum", "granularity": "monthly", "table": "store_est_t_m_fact"},
            {"agg": "sum", "granularity": "quarterly", "table": "store_est_t_q_fact"},
            {"agg": "sum", "granularity": "yearly", "table": "store_est_t_y_fact"}
        ]
    },
    "action_on_conflict": "UPDATE"
};






def dateRange(beginDate, endDate):
    dates = []
    dt = datetime.datetime.strptime(beginDate, "%Y-%m-%d")
    date = beginDate[:]
    while date <= endDate:
        dates.append(date)
        dt = dt + datetime.timedelta(1)
        date = dt.strftime("%Y-%m-%d")
    return dates


def main(spark, params):
    start = params.get("start")
    end = params.get("end")
    partition_nums = params.get("nums", 27)
    urn = Urn(
        namespace='aa.store.est.v3',
        identifier=atomic_id()
    )
    start = datetime.datetime.strptime(start, "%Y-%m-%d")
    end = datetime.datetime.strptime(end, "%Y-%m-%d")
    prepare_table(spark, urn,storage_definition, start,end )

def prepare_table(spark, urn, storage_definition, start, end):
    storage_definition['schema'] = 'store'
    storage_definition['temp_table'] = storage_definition['table']
    _validate_storage_definition(urn.namespace, storage_definition)
    config = _db_config(urn, storage_definition)
    if not _is_base_table_exist(config, storage_definition):
        _create_base_table(config, storage_definition)
    _create_index(config, storage_definition)
    if storage_definition.get('partition_by') is not None:
        _create_partition_table(config, storage_definition, _gen_partition_df(spark, start, end, storage_definition))


def _gen_partition_df(spark, start, end, storage_definition):
    column = storage_definition['partition_by']['name']
    rows = []
    if storage_definition['partition_by']['type'] == 'range':
        p_start = start
        p_end = end
        for date in daterange(p_start, p_end):
            row = Row(_1=str_date(date))
            rows.append(row)
    else:
        return None
    return spark.createDataFrame(rows, [column])


main(spark, {"start":"2017-01-01", "end":"2017-12-31"})


In [0]:

import datetime
def dateRange(beginDate, endDate):
    dates = []
    dt = datetime.datetime.strptime(beginDate, "%Y-%m-%d")
    date = beginDate[:]
    while date <= endDate:
        dates.append(date)
        dt = dt + datetime.timedelta(1)
        date = dt.strftime("%Y-%m-%d")
    return dates
print dateRange("2017-01-01","2017-01-31")